# Code to run website tests
### The model parameters are already trained using the Training Visualization 

In [ ]:
pre_i=1
post_i=2
bw_i=3
bf_i=4

In [ ]:
import csv
import matplotlib.pyplot as plt
import sys
import math

import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq
from scipy.fft import irfft
import numpy as np
import statistics
from statistics import mean, pstdev
import pandas as pd

yellow = '\033[93m'
green = '\033[92m'
red = '\033[91m'
blue = '\033[94m'
pink = '\033[95m'
black = '\033[90m'

def smoothen(time, data, rtt):
    # Smoothening 
    left = 0
    right = 0
    run_sum = 0
    avg_data = []
    new_time = []
    roll_time = time
    roll_data = data
    while right < len(roll_time):
        while(right < len(roll_time) and (roll_time[right]-roll_time[left] < 2*rtt)):
            run_sum+=roll_data[right]
            right+=1
        new_time.append(float(roll_time[right-1]+roll_time[left])/2)
        avg_data.append(float(run_sum)/(right-left))
        run_sum-=roll_data[left]
        left+=1
    return new_time, avg_data


def get_fft(data):
    n = len(data)
    data_step = 0.002
    yf = rfft(data)
    xf = rfftfreq(n,data_step)
    return yf,xf

def get_fft_smoothening(data, time, ax,rtt,p):
    rtt=rtt
    yf, xf = get_fft(data)
    thresh  = (1/rtt)
    thresh_ind = 0
    for i in range(len(xf)) :
        freq = xf[i]
        if(freq > thresh):
            thresh_ind = i
            break
            
    yf_clean = yf
    yf_clean[thresh_ind+1:] = 0
    new_f_clean = irfft(yf_clean)
    start_len = len(time) - len(new_f_clean)

    plot_data = new_f_clean
#     if p=="y":
#         ax.plot(time[start_len:], plot_data, 'k', label='FFT smoothening', linewidth=1.5)

    plot_time = time[start_len:] 
    return plot_time, plot_data

def plot_d(ax, time, data, c, l, alpha=1):
    ax.plot(time, data, color=c, lw=2, label = l,alpha=alpha)


def plot_one_bt(f, p,t=1):
#     print(f)
    file_name = f.split("/")[-1]
    fs = file_name.split("-")

    pre = int(fs[1])
    post = int(fs[2])
    rtt = float(((pre+post)*2))/1000
    ax = 0
    if t==1:
        data, time, retrans = get_window(f,"n",t)
    elif t==2:
        data, time, retrans, OOA, DA = get_window(f,"n",t)
    if p == 'y':
        fig, ax = plt.subplots(1,1, figsize=(15,8))
        for t in retrans :
            plt.axvline(x = t, color = 'm',alpha=0.5)
        if t == 2:
            for t in OOA :
                plt.axvline(x = t, color = 'k', lw=2)
            for t in DA:
                plt.axvline(x = t, color = 'g', lw=0.5, alpha = 0.5)
        plot_d(ax,time,data, "r","Original")
    time, data = get_fft_smoothening(data, time, ax,rtt,"y")

    #         plot_d(ax,time,data,"b","FFT Smoothened" )
#         print(len(time), len(data))
    time, data = smoothen(time, data, rtt)
#         print(len(time), len(data))
    if p == 'y':
        plot_d(ax, time, data, "b", "Smoothened",alpha=0.5)
        ax.legend()
#             plt.savefig("./plots/"+f+".png")
        plt.show()
#     return time, data, grad_time, grad_data, rtt
#     print("Black : OOA, Green : DA, Magenta : RP")
    return time, data, retrans, rtt 


def get_time_features(retrans,time,rtt):
    time_thresh = 20*rtt
    features = []
    for i in range(1, len(retrans)):
        if retrans[i]-retrans[i-1] >= time_thresh:
            features.append([retrans[i-1], retrans[i]])
    # add a feature that finished when the experiment ends
    if len(retrans)>0:
        if time[-1] - retrans[-1] > 20*rtt :
            features.append([retrans[-1],time[-1]])
    return features

def get_features(time, features):
    left = 0
    right = 0
    feature_index = 0
    in_feature = 0
    index_features = []
    while right < len(time) and feature_index < len(features): 
        if in_feature == 0 and time[right]>=features[feature_index][0]:
            in_feature = 1
            left = right
        elif in_feature == 1 and time[right] > features[feature_index][1]:
            in_feature = 0
            index_features.append([left, right-1])
            feature_index+=1
        right+=1
    if in_feature == 1:
        index_features.append([left, right-1])
    return index_features

def get_plot_features(curr_file, p):
    time, data, retrans, rtt = plot_one_bt(curr_file,p=p,t=1)
    time_features = get_time_features(retrans,time,rtt)
    features = get_features(time, time_features)    
    if p == 'y':
        fig, ax = plt.subplots(1,1, figsize=(15,8))
        plot_d(ax, time, data, "b", "Smoothened")
        for ft in features : 
#             print(time[ft[1]]-time[ft[0]])
            ax.plot(time[ft[0]:ft[1]+1], data[ft[0]:ft[1]+1], color = 'r')
        plt.show()
    return time, data, features


SHOW=True
MULTI_GRAPH=False
SMOOTHENING=False
ONLY_STATS=False
s_factor=0.9


PKT_SIZE = 88


'''
TODO: 
o Add functionality where you only plot flows that send more than x bytes of data
o Sort stats and graphs by flow size
o Organize plots by flow size (larger flows have larger graphs)
o Custom smoothening function
'''

fields=["time", "frame_time_rel", "tcp_time_rel", "frame_num", "frame_len", "ip_src", "src_port", "ip_dest", "dest_port", "tcp_len", "seq", "ack"]

class pkt:
    contents=[]
    def __init__(self, fields) -> None:
        self.contents=[]
        for f in fields:
            self.contents.append(f)

    def get(self, field):
        return self.contents[fields.index(field)]
        

def process_flows(cc, dir,p="y"):
    name = dir+cc+"-tcp.csv"
    with open(name) as csv_file:
        print("Reading "+name+"...")
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        total_bytes=0
        '''
        Flow tracking:
        o Identify all packets that are either sourced from or headed to 100.64.0.2
        o Group different flows by client's port
        '''
        flows={}
        data_sent=0
        # ACK and RTX measurement
        ooa = set()
        rp = set()
        ooaCount=0
        rpCount=0
        daCount=0
        backAck=0
        for row in csv_reader:
            reTx=0
            
            ooAck=0
            dupAck=0
            retPacket=0
            
            packet=pkt(row)
            ackPkt=False
            validPkt=False
            if line_count==0:
                # reject the header
                line_count+=1
                continue
            if data_sent == 0 : 
                if "100.64.0." in packet.get("ip_src"):
                    num = int(packet.get("ip_src")[-1])
                    if num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_src")
                if "100.64.0." in packet.get("ip_dest"):
                    num = int(packet.get("ip_dest")[-1])
                    if num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_dest")
                if data_sent == 0:
                    continue
            if packet.get("ip_src")==host_port and packet.get("frame_time_rel")!='' and packet.get("ack")!='': 
                # we care about this ACK packet
                validPkt=True
                ackPkt=True
                port=packet.get("src_port")
                #PORTCHECK
#                 if int(port) != 50468:
#                     continue
                if port not in flows:
                    flows[port]={"OOA":[],"DA":[],"max_seq":0,"loss_bif":0,"max_ack":int(packet.get("ack")),"serverip":packet.get("ip_dest"), "serverport":packet.get("dest_port"), "act_times":[],"times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0, "drop":[], "next":0, "retrans":[]}
                else:
                    # check for Out of Order Ack (OOA)
                    if int(packet.get("ack")) <= int(flows[port]["max_ack"]):
                        if int(packet.get("ack")) == backAck :
                            dupAck = True
                            flows[port]["DA"].append(float(packet.get("frame_time_rel")))
                        else :
                            ooAck = True
                            flows[port]["OOA"].append(float(packet.get("frame_time_rel")))
                        backAck = int(packet.get("ack"))
                    # update max_ack
                    flows[port]["max_ack"] = max(flows[port]["max_ack"], int(packet.get("ack")))
                    if int(packet.get("seq")) < flows[port]["max_ack"]:
                        reTx += int(packet.get("tcp_len"))
#                     flows[port]["times"].append(float(packet.get("frame_time_rel")) )
                    
            elif packet.get("ip_dest")==host_port and packet.get("frame_time_rel")!='' and packet.get("seq")!='':
                #we care about this Data packet
                validPkt=True
                port=packet.get("dest_port")
                #PORTCHECK
#                 if int(port) != 50468:
#                     continue
                seq=int(packet.get("seq"))
                tcp_len=int(packet.get("tcp_len"))
                if port not in flows:
                    flows[port]={"OOA":[],"DA":[],"max_seq":int(packet.get("seq")),"loss_bif":0,"max_ack":0,"serverip":packet.get("ip_src"), "serverport":packet.get("src_port"),"act_times":[], "times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0, "drop":[], "next":0, "retrans":[]}
                
                else:
                    flows[port]["max_seq"] = max(flows[port]["max_seq"], int(packet.get("seq")))
                
                
                if int(packet.get("seq")) < flows[port]["max_seq"] :
                    retPacket = True
                    flows[port]["retrans"].append(flows[port]["times"][-1])
                    
            if validPkt==True:
                bif = 0
                normal_est_bif = int(flows[port]["max_seq"]) - int(flows[port]["max_ack"]) + PKT_SIZE#+ reTx
                loss_est_bif = flows[port]["loss_bif"]
                if ackPkt and dupAck and len(flows[port]["windows"]) > 10:
                    if dupAck:
                        # if we have received a duplicate ack then we need to reduce the bytes in flight by packet size
                        # we also increase max ack to correct for the consolidated ack being sent later
                        loss_est_bif = int(flows[port]["windows"][-1]) - PKT_SIZE
                        flows[port]["max_ack"] += PKT_SIZE
                        
                        bif = min( normal_est_bif, loss_est_bif)
                        if p == "y" :
                            print(green+"Duplicate Ack",int(packet.get("ack")),"Max Ack",flows[port]["max_ack"],"BIF",bif)
                        
#                     elif ooAck:
#                         # first out of order ack that we have recieved not a duplicated ack
#                         # the reason would be restransimitted packet so dont need to correct for this
                        
                elif ackPkt :
                    loss_est_bif = normal_est_bif 
                    bif = normal_est_bif    
                    if ooAck :
                        ooaCount+=1
                        if p == "y":
                            print(red+"Out of Order Ack",int(packet.get("ack")),"Max Ack",flows[port]["max_ack"],"BIF",normal_est_bif)
                        ooa.add(int(packet.get("ack")))
                    else:
                        if p == "y":
                            print(black+"Inorder Ack",int(packet.get("ack")),"Max Seq",flows[port]["max_seq"],"BIF",normal_est_bif)
                else :
                    bif = normal_est_bif
                    if retPacket==True:
                        rpCount+=1
                        rp.add(int(packet.get("seq")))
                        if p == "y":
                            print(pink+"Retransmitted Packet",int(packet.get("seq")), "Next", flows[port]["max_seq"]+PKT_SIZE, "BIF",bif)
                    else :
                        if p == "y":
                            print(blue+"Inorder Packet", int(packet.get("seq")), "Next", flows[port]["max_seq"]+PKT_SIZE, "BIF",bif)
                flows[port]["loss_bif"] = loss_est_bif
                flows[port]["windows"].append( int(bif) )
                flows[port]["times"].append( float(packet.get("frame_time_rel")) )
                
#                 if ackPkt and dupAck and len(flows[port]["windows"]) > 10: # we have received atleast the first window
# #                     if len(flows[port]["windows"]) < 2000: # print reTx in first 200 packets
# #                         print( packet.get("ack"), flows[port]["max_ack"])
#                     loss_est_bif = int(flows[port]["windows"][-1]) - PKT_SIZE
#                     flows[port]["max_ack"] += PKT_SIZE
#                     bif = min( normal_est_bif, loss_est_bif )
#                 elif ackPkt:
#                     loss_est_bif = normal_est_bif 
#                     bif = normal_est_bif
#                 else:
#                     bif = normal_est_bif
#                 flows[port]["loss_bif"] = loss_est_bif
#                 flows[port]["windows"].append( int(bif) )
            
            
            line_count+=1
            total_bytes+=int(packet.get("frame_len"))
            #print(line_count, total_bytes)
            
#         print("total bytes processed:", total_bytes/1000, "KBytes for", cc, "(unlimited)")
        if p == "y":
            print("Out of Order Acks",len(ooa),"Retransmitted Packets",len(rp))
            print("Count Out of Order Acks",ooaCount,"Retransmitted Packets",rpCount)
            print("OOA",ooa,"RP",rp)
    return flows

def custom_smooth_function():
    pass

def get_flow_stats(flows):
    num=len(flows.keys())
    print("FLOW STATISTICS: \nNumber of flows: ", num)
    print("------------------------------------------------------------------------------")
    print('%6s'%"port", '%15s'%"SrcIP", '%8s'%"SrcPort",  '%8s'%"duration",  '%8s'%"start",  '%8s'%"end", '%8s'%"Sent (B)", '%8s'%"Recv (B)",)
    for k in flows.keys():
        print('%6s'%k, '%15s'%flows[k]["serverip"], '%8s'%flows[k]["serverport"], '%8s'%str('%.2f'%(flows[k]["times"][-1]-flows[k]["times"][0])), '%8s'%str('%.2f'%flows[k]["times"][0]), '%8s'%str('%.2f'%flows[k]["times"][-1]), '%8s'%flows[k]["last_seq"], '%8s'%flows[k]["last_ack"])
        #print("    * Flow "+str(k)+": ", flows[k]["last_ack"], " ", flows[k]["last_seq"], " bytes transfered.")
    return num

def run(files):
    flows = {}
    for f in files:
        algo_cc=f
        #Get the data for all the flows
        print("==============================================================================")
        print("opening trace ../measurements/"+algo_cc+".csv...")
        flows = process_flows(algo_cc, "./Nebby/measurements-new-btl/50-200-2-60/")
        #decide on final graph layout
        num = get_flow_stats(flows)

        if ONLY_STATS:
            sys.exit()

        if num==1:
            MULTI_GRAPH=False
        #grid size
        if MULTI_GRAPH:
            size=(0,0)
            grids={1:(2,2), 2:(2,2), 4:(2,2), 6:(2,3), 9:(3,3), 12:(3,4), 15:(3,5), 16:(4,4), 20:(5,4), 24:(6,4), 30:(6,5), 36:(6,6), 40:(8,5), 42:(8,7), 49:(7,7)}
            g=num
            while g<=49 and g not in grids:
                g+=1
            if g in grids.keys():
                size=grids[g]
            else:
                size=grids[49]  
            fig, axs = plt.subplots(size[0], size[1])
            for i in range(size[0]):
                for j in range(size[1]):
                    #axs[i][j].legend(loc="lower right")
                    if i==size[0]-1:
                        axs[i][j].set_xlabel("Time (s)")
                    if j==0:
                        axs[i][j].set_ylabel("Bytes in flight")
        else:
            plt.xlabel("Time (s)")
            plt.ylabel("Bytes in flight")
        counter=0
        for port in flows.keys():
            if MULTI_GRAPH:  
                axs[counter%size[0]][(counter//size[0])%size[1]].scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
                axs[counter%size[0]][(counter//size[0])%size[1]].plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
            else:
                plt.plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
                plt.scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
            counter+=1
        if MULTI_GRAPH:
            counter=0
            for port in flows.keys():
                axs[counter%size[0]][(counter//size[0])%size[1]].legend()
                counter+=1
        else:
            plt.legend()
        if MULTI_GRAPH:
            fig.set_size_inches(16, 12)
        if SHOW:
            plt.show()
        else:
            plt.savefig("../logs/results/"+algo_cc+".png", dpi=600, bbox_inches='tight', pad_inches=0)
    return flows


def split_path(f):
    path = f.split("/")
    file_name = path[-1][:-8]
    folder_path = "/".join(path[:-1])
    folder_path = folder_path + "/"
    algo_cc = file_name
    return algo_cc, folder_path

def get_window(f,p,t=1):
    algo_cc, folder_path = split_path(f)
    flows = process_flows(algo_cc, folder_path,p=p)
#     flows = process_flows(algo_cc, "../measurements/m/",p=p)
#     flows = process_flows(algo_cc, "./Nebby/measurements/",p=p)    
#     flows = process_flows(algo_cc, "./Nebby/measurements-new-btl/50-200-2-60/",p=p)
#     flows = process_flows(algo_cc, "./Nebby/measurements-100-150/",p=p)
    params = algo_cc.split("-")
    data = []
    time = []
    drops = []
    retrans = []
    OOA = []
    DA = []
    use_port = 0
    maxx = 0
    print("All Ports : ", flows.keys())
    for port in flows.keys():
        if len(flows[port]['windows']) > maxx:
            maxx = len(flows[port]['windows'])
            use_port = port
    print("Port",use_port)
    data = flows[use_port]['windows']
    time = flows[use_port]['times']
    retrans = flows[use_port]['retrans']
    OOA = flows[use_port]['OOA']
    DA = flows[use_port]['DA']
    if p == "y":
        plt.plot(time, data)
#     time_index = len(time)-1
#     for index in range(len(flows[use_port]['times'])-1):
#         if flows[use_port]['times'][index+1] - flows[use_port]['times'][index] > :
#             time_index = index
#     time_last = flows[use_port]['times'][time_index]
#     data = data[:time_index+1]
#     time = time[:time_index+1]
    if t==2:
        return data, time, retrans, OOA, DA
    if t==1:
        return data, time, retrans


def getProbes(time, data, rtt, bdp, bw=200):
    if bw==200:
        thresh = 8*rtt
        st_thresh = 0.025
        error = 0.08
        alpha = 1.10
    if bw==1000:
        thresh = 4*rtt
        st_thresh = 0.025
        error = 0.02
        alpha = 1
    probe_index = []
    left = 0
    right = 0
    bdp_thresh = bdp/2
    prev_right = 0
    end = 0
    while right < len(data):
        while right < len(data) and (time[right]-time[left]) < thresh:
            right+=1
        if right == len(data):
            end = 1
            right-=1
        mid = math.floor(left + (right - left)/2)
        mid_val = 0
        left_mid = mid
        right_mid = mid
#         print(left,right)
#         while left_mid > 0 and abs(time[mid]-time[left_mid])<rtt:
#             if data[left_mid] > data[left] and data[left_mid] > data[right]:
#                 mid_val+=1
#             left_mid-=1
#         while right_mid < len(data)-1 and abs(time[right_mid]-time[mid])<rtt:
#             if data[right_mid] > data[left] and data[right_mid] > data[right]:
#                 mid_val+=1
#             right_mid+=1
#         total = right_mid - left_mid
#         peak = 0
#         if round(float(mid_val)/total,2) > 0.99 :
#             print(round(float(mid_val)/total,2))
#             peak=1
        go = 0
        if data[mid] > data[left] and data[mid] > data[right]:
#         if peak == 1:
            #this  is  peak
#             go = 1
            t_l = left
            t_r = right
            while(t_l > 0 and time[left]-time[t_l] < thresh/2):
                t_l-=1
            while(t_r < len(data)-1 and time[t_r]-time[right] < thresh/2):
                t_r+=1
            left_sd = round(np.std(data[t_l:left])/(bdp_thresh*2),3)
            right_sd = round(np.std(data[right:t_r])/(bdp_thresh*2),3)
            if float(abs(data[left]-data[right]))/data[left] < error:
                # this has the left and right points not too different from each other
                side_avg = float((data[left]+data[right]))/2
                local_max = max(data[left:right+1])
#                 go = 1
#                 print("Yes")
                if float(local_max)/side_avg > alpha and local_max > bdp_thresh: 
                    # this means that the peak is quite steep
#                     print("This")
#                     go = 1
                    if (left_sd < st_thresh) and (right_sd < st_thresh):
                        # this means that the lest and right are quite stable respectvely
                        go = 1
#                         print("Out", left_sd, right_sd, st_thresh)
        if go == 1: 
            try :
                probe_index.append([left,right,float(local_max)/side_avg, time[right]-time[left], left_sd, right_sd, t_l,t_r])
            except :
                probe_index.append([left,right])
            #Once you have found something you directly move past it
            left = right-1
        if end:
            right+=1
        left+=1
    return probe_index


def checkBBR(files,p="n"):
    classi = []
    for f in files:
        file_name = f.split("/")[-1]
        para = file_name.split("-")
        rtt = int(para[2])*2
        bw = int(para[3])
        bf = int(para[4])
        bdp = float(bw*rtt*bf)/8
        if bw == 1000 :
            l=5
            r=15
        if bw == 200:
            l=10
            r=20
        try:
            time, data, retrans,rtt = plot_one_bt(f,p="n",t=1)
            probe_index = getProbes(time, data, rtt, bdp, bw)
            if p=="y":
                print_red(time, data, probe_index)
            prev = 0
            time_dis = []
            for p in probe_index:
                curr_ind = data.index(max(data[p[0]:p[1]+1]))
                if curr_ind > p[1] or curr_ind < p[0]:
                    print("Something Wrong")
                if prev != 0:
                    time_dis.append(abs(time[curr_ind]-prev))
                prev = time[curr_ind]
#             print(time_dis)
            count = 0
            isBBR = 0
            for t in time_dis : 
                if  t > l*rtt and t < r*rtt :
                    count+=1
                    if count >= 2: # there are three peaks consecutively
                        isBBR=1
                        break
                else:
                    count = 0
            if isBBR:
                classi.append("YES BBR")
            else:
                if len(probe_index) <= 1 :
                    classi.append("NO BBR")
                else:
                    classi.append("MAYBE BBR")
        except Exception as ex:  
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            new_message = "NC " + message
            classi.append(new_message)
    return classi

def print_red(time,data,probe_index):
    fig, ax = plt.subplots(1,1, figsize=(15,8))
    offset=0
    ax.plot(time[offset:],data[offset:])
    # for t in retrans :
    #     if t > time[offset]:
    #         plt.axvline(x = t, color = 'm',alpha=0.5)
    # ax.plot(new_time[offset:], new_data[offset:])
    for p in probe_index:
        ax.plot(time[p[0]:p[1]+1], data[p[0]:p[1]+1], color='r', lw=2)
    plt.show()

def getDivision(classi, test_files):
    yes = []
    no = []
    maybe = []
    nan = []
    for i in range(len(classi)):
        if classi[i] == "YES BBR":
            yes.append(test_files[i])
        elif classi[i] == "NO BBR":
            no.append(test_files[i])
        elif classi[i] == "MAYBE BBR":
            maybe.append(test_files[i])
        else:
            nan.append(test_files[i])
    return yes, no,maybe, nan

In [ ]:
import bisect
def lower_bound(arr, target):
    index = bisect.bisect_left(arr, target)
    return index

def sample_data_time(time, data, ss, m):
    curr_time, curr_data = adjust(time, data)
    tp = curr_time[len(curr_time)-1] - curr_time[0]
    step = tp/m
    samp_time = [curr_time[0] + i*step for i in range(m)]
    x = np.random.uniform(0,math.pi,ss)
    tr_x = np.cos(x)
    tr_x += 1
    tr_x *= (m-1)/2
    ind = [int(round(e, 0)) for e in tr_x]
    sort_ind = sorted(ind)
    tr_time = [samp_time[i] for i in sort_ind]
    new_time = []
    new_data = []
    for t in tr_time :
        i = lower_bound(curr_time, t)
        temp_t = 0
        temp_d = 0
        if round(t,6) == round(curr_time[i],6):
            temp_t = curr_time[i]
            temp_d = curr_data[i]
        else : 
            if i == 0 :
                temp_t = curr_time[i]
                temp_d = curr_data[i]
            elif i == len(curr_time)-1 :
                temp_t = curr_time[i]
                temp_d = curr_data[i]
            else :
                temp_t = (curr_time[i-1] + curr_time[i])/2
                temp_d = (curr_data[i-1] + curr_data[i])/2
        new_time.append(temp_t)
        new_data.append(temp_d)
    new_data.insert(0,curr_data[0])
    new_data.append(curr_data[-1])
    new_time.insert(0,curr_time[0])
    new_time.append(curr_time[-1])
#     return curr_time, curr_data
    return new_time, new_data

def adjust(time, data):
    try :
        start = data.index(min(data[:int(len(data)/2)]))
        end = data.index(max(data[int(len(data)/2):]))
    except :
        start = 0 
        end = len(data) - 1
    print("Difference in max and min ", end-start)
    if end - start <= 0: 
        return time, data
    new_time = time[start:end+1]
    new_data = data[start:end+1]
    return new_time, new_data

# Taking 100 as the threshold is fine 
# Now we have to see how the graphs look if we use it. 
# var = ["reno", "cubic", "bbr"]
def getRed_R(files,ss=125,p="y", ft_thresh=100):
    results = []
    for curr_file in files : 
        print(curr_file)
        file, folder_path = split_path(curr_file)
        f_split = file.split("-")
        v = f_split[0]
        rtt = float((int(f_split[pre_i]) + int(f_split[post_i]))*2)/1000
        bdp = float(rtt*1000*int(f_split[bw_i])*int(f_split[bf_i]))/8
        time, data, features = get_plot_features(curr_file, p=p)
        count = 1
        for ft in features : 
            if count > ft_thresh:
                break
            curr_time = time[ft[0]:ft[1]+1]
            curr_data = data[ft[0]:ft[1]+1]
            tr_time, tr_data = sample_data_time(curr_time, curr_data, ss, 1000)
            tr_time_pd = pd.DataFrame(tr_time)
            tr_data_pd = pd.DataFrame(tr_data)
            tr_time = list(tr_time_pd.rolling(25, center=True).mean().dropna()[0])
            tr_data = list(tr_data_pd.rolling(25, center=True).mean().dropna()[0])
            print("Feature Length ", len(tr_data))
            if p == "y" :
                plt.plot(curr_time, curr_data, c='b', alpha = 0.5, lw = 5)
                plt.plot(tr_time, tr_data, c='r', alpha = 1)
                plt.scatter(tr_time, tr_data, c='k')
#                 plt.scatter(tr_time, tr_data, c='r', s=10)
                plt.title(v)
                plt.show()
            results.append(
                {v+"_"+"data"+"_"+str(count):tr_data,
                     v+"_"+"time"+"_"+str(count):tr_time,
                        v+"_"+"rtt"+"_"+str(count):rtt, 
                             v+"_"+"bdp"+"_"+str(count):bdp})
            count+=1
    return results

def getRed(files,ss=125,p="y", ft_thresh=100):
    results = []
    for file in files :   
        f_split = file.split("-")
        v = f_split[0] + "-" + f_split[1]
        rtt = float((int(f_split[2]) + int(f_split[3]))*2)/1000
        bdp = float(rtt*1000*int(f_split[4])*int(f_split[5]))/8
        print(file)
        print("RTT",rtt,"BDP",bdp)
        time, data, features = get_plot_features(file, p=p)
        count = 1
        for ft in features : 
            if count > ft_thresh:
                break
            curr_time = time[ft[0]:ft[1]+1]
            curr_data = data[ft[0]:ft[1]+1]
            tr_time, tr_data = sample_data_time(curr_time, curr_data, ss, 1000)
            tr_time_pd = pd.DataFrame(tr_time)
            tr_data_pd = pd.DataFrame(tr_data)
            tr_time = list(tr_time_pd.rolling(25, center=True).mean().dropna()[0])
            tr_data = list(tr_data_pd.rolling(25, center=True).mean().dropna()[0])
            print("Feature Length ", len(tr_data))
            if p == "y" :
                plt.plot(curr_time, curr_data, c='b', alpha = 0.5, lw = 5)
                plt.plot(tr_time, tr_data, c='r', alpha = 1)
                plt.scatter(tr_time, tr_data, c='k')
#                 plt.scatter(tr_time, tr_data, c='r', s=10)
                plt.title(v)
                plt.show()
            results.append(
                {v+"_"+"data"+"_"+str(count):tr_data,
                     v+"_"+"time"+"_"+str(count):tr_time,
                        v+"_"+"rtt"+"_"+str(count):rtt, 
                             v+"_"+"bdp"+"_"+str(count):bdp})
            count+=1
    return results
# results = getRed(var)

def get_degree_all(time,data, p="n", max_deg=3):
    p_net = []
    mse_l = []
    fit_net = []
    for d in range(1,max_deg+1):
        p_temp = np.polyfit(time,data,d)
        p_net.append(p_temp)
        fit_net.append(np.polyval(p_temp,time))
        mse_l.append(mse(data,fit_net[-1]))
    if p =='y':
#         print("1 ", p1, "MSE ", mse(data, fit_l))
        plt.plot(time, data,c='k',label='Truth')
#         plt.plot(time, fit_l)
        for d in range(0, max_deg):
            plot_label = "degree"+str(d+1)
            plt.plot(time, fit_net[d],label="degree" + str(d+1))
        plt.legend()
        plt.show()
    return max_deg,p_net, mse_l

MAX_DEG=3
from sklearn.metrics import mean_squared_error as mse
def get_degree(time,data, p="n", max_deg=MAX_DEG):
    p_net = []
    mse_l = []
    fit_net = []
    for d in range(1,max_deg+1):
        p_temp = np.polyfit(time,data, d)
        p_net.append(p_temp[0:-1])
        fit_net.append(np.polyval(p_temp,time))
        mse_l.append(mse(data,fit_net[-1]))
    if p =='y':
#         print("1 ", p1, "MSE ", mse(data, fit_l))
        plt.plot(time, data,c='k',label='Truth')
#         plt.plot(time, fit_l)
        for d in range(max_deg-1, max_deg):
            plot_label = "degree"+str(d)
            plt.plot(time, fit_net[d],label="degree" + str(d+1))
        plt.legend()
        plt.show()
    return max_deg,p_net[max_deg-1], mse_l

def normalize(time, data, rtt, bdp):
    new_time = time - min(time)
    new_data = data - min(data)
    new_time = (new_time/max(new_time))*10
    new_data = (new_data/max(new_data))*10    
    return new_time, new_data
    
from statistics import mean 
def get_feature_degree_R(files,ss=225,p='n',ft_thresh=3,max_deg=MAX_DEG):
    results = getRed_R(files,ss,p=p,ft_thresh=ft_thresh)
    count_features = 0
    mp = {}
    for item in results :
        for ele in list(item.keys()):
            name_list = ele.split("_")
            website = name_list[0]
            if "data"==name_list[1] :
                curr_data = np.array(item[ele])
            if "time"==name_list[1] :
                curr_time = np.array(item[ele])
            if "rtt"==name_list[1] :
                curr_rtt = item[ele]
            if "bdp"==name_list[1] :
                curr_bdp = item[ele]
#         print(website)
#         print(curr_data, curr_time)
        curr_time, curr_data = normalize(curr_time, curr_data, curr_rtt, curr_bdp)
        count_features += 1
        max_deg,p_net,mse_l = get_degree_all(curr_time, curr_data,p=p,max_deg=max_deg)
        mp[website] = {
        'data':curr_data,
        'time':curr_time,
        "max_deg":max_deg,
        "p_net":p_net,
        "mse_l":mse_l
    }
    return mp

from statistics import mean 
def get_feature_degree(files,ss=225,p='n',ft_thresh=3,max_deg=MAX_DEG):
    results = getRed(files,ss,p=p,ft_thresh=ft_thresh)
    errors = []
    mp = {}
    cc_mp = {}
    count_features = 0
    for item in results :
        for ele in list(item.keys()):
            name_list = ele.split("_")
            cc = name_list[0]
            name = name_list[0] + name_list[-1]
            if "data" in ele :
                curr_data = np.array(item[ele])
            if "time" in ele :
                curr_time = np.array(item[ele])
            if "rtt" in ele :
                curr_rtt = item[ele]
            if "bdp" in ele :
                curr_bdp = item[ele]
        curr_time, curr_data = normalize(curr_time, curr_data, curr_rtt, curr_bdp)
        count_features += 1
        print("Name :",name)
        degree, coeff, error_item = get_degree(curr_time, curr_data,p=p,max_deg=max_deg)
        # Adding time feature here
#         coeff_list = list(coeff)
#         coeff_list.append(abs(curr_time[-1]-curr_time[0]))
#         coeff = np.array(coeff_list)
#         print(coeff)
        #Adding new for scalalble and yeah
        cc_curr = cc.split("-")[0]
        # Adding another check for scalable and yeah
#         if cc_curr in ['scalable','yeah']:
#             # Look at the fifth coefficient
#             if round(coeff[0],6) < 0.000015:
#                 print(coeff[0])
#                 continue
#         coeff.append(abs(curr_time[-1]-curr_time[0]))
        mp[name] = {'d':degree, 'coeff':coeff, 'error':error_item, 'data':curr_data, 'time':curr_time}
        if cc not in cc_mp :
            cc_mp[cc] = []
        cc_mp[cc].append(mp[name])
    return cc_mp

from scipy.stats import multivariate_normal as mvn
def getPDensity(curr, cc_gaussian_params):
    prob = {}
    for cc in cc_gaussian_params:
        mn = cc_gaussian_params[cc]['mean']
        covar = cc_gaussian_params[cc]['covar']
#         print("CC being checked against",cc)
#         print(np.linalg.det(covar))
        curr_p = mvn.pdf(curr,mean=mn, cov=covar, allow_singular=True)
        prob[cc]=curr_p
    return prob

In [ ]:
def getBestDegree(nmp):
    results = {}
    too_much_error = {}
    for name in nmp.keys():
        print(name)
#         curr_time, curr_data, retrans, rtt = plot_one_bt(name+"-0-50-200-2-aws-88-60",'y',1)
        data = nmp[name]['data']
        time = nmp[name]['time']
        max_deg = nmp[name]['max_deg']
        p_net = []
        for coeff in nmp[name]['p_net']:
            temp_pnet = [c for c in coeff]
            p_net.append(temp_pnet)
        mse_l = nmp[name]['mse_l']
        plt.plot(time, data,c='k',label='Truth')
#         plt.plot(time, fit_l)
        for d in range(0, max_deg):
            plot_label = "degree"+str(d+1)
            fit_net = np.polyval(p_net[d],time)
            plt.plot(time, fit_net,label="degree" + str(d+1))
        plt.legend()
        plt.show()

        names = []
        loss = []
        lambd = 0.02
        for i in range(len(mse_l)):
            loss.append((i+1)*sum(p_net[i])*lambd)
        print("loss", loss)
        print("mse", mse_l)
        for d in range(0, max_deg):
            names.append("degree "+str(d+1))
        plt.bar(names,mse_l,color='blue',width=0.4,label="mse")
        plt.bar(names,loss,bottom=mse_l,color='maroon',width=0.4,label="reg_loss")

        plt.legend()
        plt.show()
        errors = []
        # The code for deciding the categories
        for i in range(0,max_deg):
            errors.append(loss[i]+mse_l[i])
        deg = errors.index(min(errors))+1
        
        # Error threshold
        error_threshold = 1
        if errors[deg-1] > error_threshold:
            current_cc = name.split("-")[0]
            too_much_error[current_cc]={
                'deg':deg,
                'coeff':p_net[deg-1],
                'error':errors[deg-1]
            }
            continue
        
    #     if deg < 3:
    #         deg = mse_l.index(min(mse_l[0:2]))+1
    #     print(deg)
        current_cc = name.split("-")[0]
        results[current_cc]={
            'deg':deg,
            'coeff':p_net[deg-1],
            'error':errors[deg-1]
        }
    return results, too_much_error

In [ ]:
from scipy.stats import multivariate_normal as mvn
def getPDensityR(curr, cc_gaussian_params):
    curr_coeff = curr[:-1]
    prob = {}
    for cc in cc_gaussian_params:
        mn = cc_gaussian_params[cc]['mean']
        covar = cc_gaussian_params[cc]['covar']
        if len(curr_coeff) == len(mn):
            print(cc)
            print(curr_coeff)
            print(mn)
            print(covar)
            curr_p = mvn.pdf(curr_coeff,mean=mn, cov=covar, allow_singular=True)
            print(curr_p)
            prob[cc]=curr_p
#         print("CC being checked against",cc)
#         print(np.linalg.det(covar))
    return prob

def getWebDensity(mp, cc_gaussian_params):
    acc_w = {}
    for web in mp:
        p_dense = getPDensityR(mp[web]['coeff'], cc_gaussian_params)
        if web not in acc_w:
            acc_w[web]={}
        ccs = np.array(list(p_dense.keys()))
        vals = np.array(list(p_dense.values()))
        
        p_ind = list(np.argsort(vals))
        p_ind.reverse()
        acc_w[web]['ccs'] = [ccs[i] for i in p_ind]
        acc_w[web]['density'] = np.array([vals[i] for i in p_ind])
        maxx = max(acc_w[web]['density'])
        minn = min(acc_w[web]['density'])
        acc_w[web]['relative'] = (acc_w[web]['density']-minn)/(maxx-minn) 
    return acc_w

def predictCC(acc_w):
    pred = {}
    for web in acc_w:
        if acc_w[web]['density'][0] < 0.01:
            pred[web]='new'
        elif acc_w[web]['relative'][1] > 0.01:
            pred[web]='confused'
        else :
            pred[web]=acc_w[web]['ccs'][0]
    return pred, acc_w

def getPredictions(cc_mp,cc_gp,no):
    acc_w = getWebDensity(cc_mp, cc_gp)
    pred, acc_w = predictCC(acc_w)
    predictions = {}
    no_feature = []
    for f in no:
        tp_mp = get_feature_degree_R([f],ss=225,p="y",ft_thresh=1,max_deg=3)
        if len(cc_mp.keys()) == 0:
            continue
        new_f = f.split("/")[-1]
        w = new_f.split("-")[0]
        if w not in acc_w.keys():
            print("No Feature")
            no_feature.append(w)
            predictions[w]="No feature"
        else:
            print(cc_mp[new_f.split("-")[0]]['coeff'])
    #         time, data, retrans, rtt= plot_one_bt(f,"y",1)
            predictions[w] = {}
            print("Top 5 CCS :", acc_w[w]['ccs'][0:5])
            predictions[w]['ccs'] = acc_w[w]['ccs'][0:5]
            
            print("Density Values:",acc_w[w]['density'][0:5])
            predictions[w]['density'] = acc_w[w]['density'][0:5]
            
            print("Relative Density Values:",acc_w[w]['relative'][0:5])
            predictions[w]['relative'] = acc_w[w]['relative'] = acc_w[w]['relative'][0:5]
            
            print("Prediction:",pred[w])
            predictions[w]['final'] = pred[w]
    return predictions

In [ ]:
import pickle
with open('cc_gp_cubicQ.txt', 'rb') as f:
    cc_gp = pickle.load(f)

# Running the same on all files

In [ ]:
folder = '../../aws/website_measurements/26_1k_cut/mumbai/'

In [ ]:
def file_filter(file):
    if "udp.csv" in file:
        return 0
    if "0-50-100-2" in file:
        return 0
    return 1
    

In [ ]:
mumbai_files = []
import os 
for f in os.listdir(folder):
    if file_filter(f) == 1:
        mumbai_files.append(folder+f)
mumbai_files = sorted(mumbai_files)

In [ ]:
classi = checkBBR(mumbai_files,p="n")

In [ ]:
yes,no,maybe,nan = getDivision(classi,mumbai_files)
print(len(yes),len(no),len(maybe),len(nan))

In [ ]:
web_mp = get_feature_degree_R(no,ss=225,p="n",ft_thresh=1,max_deg=3)

In [ ]:
web_mp

In [ ]:
web_cc_mp, too_much_error = getBestDegree(web_mp)

In [ ]:
print("The ones that have too much error", len(too_much_error.keys()))

In [ ]:
counts = [0,0,0]
for web in web_cc_mp:
    counts[web_cc_mp[web]['deg']-1]+=1
print("1,2,3")
print(counts)

In [ ]:
web_cc_mp

In [ ]:
import pickle
with open('vals_cubicQ_new.txt', 'rb') as f:
    vals_cubicQ = pickle.load(f)

In [ ]:
vals_cubicQ.pop('vegas')

In [ ]:
vals_cubicQ.pop('nv')

In [ ]:
# Not using vegas and nv

In [ ]:
cc_degree = {'bic': 1,
 'dctcp': 2,
 'highspeed': 2,
 'htcp': 3,
 'lp': 2,
 'scalable': 1,
 'veno': 3,
 'westwood': 2,
 'yeah': 1,
 'cubic': 3,
 'reno': 2,
'cubicQ':3}

In [ ]:
# Saving the cc_gaussian_params:
import pickle
with open("../analysis/final/cc_degree.txt",'wb') as f:
    pickle.dump(cc_degree,f)

# Detecting Outliers through distance measures

### Scaling the coeff for correct analysis according to degree and calculating mean 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
degrees = [1,2,3]

In [ ]:
# mm_scaled_vals = {}
# for cc in vals_cubicQ:
#     if cc == 'nv' :
#         continue
#     if cc_degree[cc] not in mm_scaled_vals:
#         mm_scaled_vals[cc_degree[cc]] = {}
#     mm_scaled_vals[cc_degree[cc]][cc] = vals_cubicQ[cc][1]

# for d in mm_scaled_vals:
#     temp_data = []
#     for cc in mm_scaled_vals[d]:
#         temp_data+=mm_scaled_vals[d][cc]
#     temp_data = np.array(temp_data)
#     min_data = np.min(temp_data)
#     max_data = np.max(temp_data)
#     for cc in mm_scaled_vals[d]:
#         mm_scaled_vals[d][cc] = (mm_scaled_vals[d][cc]-min_data)/(max_data-min_data)
#     mm_scaled_vals[d]['max_data'] = max_data 
#     mm_scaled_vals[d]['min_data'] = min_data

In [ ]:
scaled_vals = {}
for cc in vals_cubicQ:
    if cc_degree[cc] not in scaled_vals:
        scaled_vals[cc_degree[cc]] = {}
    scaled_vals[cc_degree[cc]][cc] = vals_cubicQ[cc][1]

for d in scaled_vals:
    temp_data = []
    for cc in scaled_vals[d]:
        temp_data += scaled_vals[d][cc]
    scaler = StandardScaler().fit(temp_data)
    for cc in scaled_vals[d]:
        scaled_vals[d][cc] = np.array(scaler.transform(scaled_vals[d][cc]))
        if cc == 'scalable':
            for i in range(len(scaled_vals[d][cc])):
                if scaled_vals[d][cc][i] > 10 :
                    scaled_vals[d][cc][i] = scaled_vals[d][cc][i-1]
                    
    scaled_vals[d]['scaler'] = scaler

In [ ]:
# Saving the cc_gaussian_params:
import pickle
with open("../analysis/final/vals.txt",'wb') as f:
    pickle.dump(vals_cubicQ,f)

In [ ]:
# Saving the cc_gaussian_params:
import pickle
with open("../analysis/final/scaled_vals.txt",'wb') as f:
    pickle.dump(scaled_vals,f)

### Plotting all the ccs with the websites

In [ ]:
def get_website_by_degree(d):
    labels = []
    result = []
    for web in web_cc_mp:
        if web_cc_mp[web]['deg'] == d:
            labels.append(web)
            result.append(web_cc_mp[web]['coeff'][0:d])
    return {'labels' : labels, 'data' : result}

In [ ]:
web_data = {}
scaled_web_data = {}

In [ ]:
degree = 2
scaled_vals[degree]['combined'] = []
for cc in ['dctcp','highspeed','lp','reno']:
    scaled_vals[degree]['combined'] += list(scaled_vals[degree][cc])
    scaled_vals[degree].pop(cc)
scaled_vals[degree]['combined'] = np.array(scaled_vals[degree]['combined'])

In [ ]:
scaled_vals[2].keys()

In [ ]:
# web_data = {}
# mm_scaled_web_data = {}

#### D = 1

In [ ]:
# not_cc = ['max_data','min_data']

In [ ]:
degree = 1
web_data[degree]  = get_website_by_degree(degree)
scaled_web_data[degree] = {'labels':web_data[degree]['labels'], 'data':np.array(scaled_vals[degree]['scaler'].transform(web_data[degree]['data']))}

fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(12,5))
for cc in scaled_vals[degree]:
    if cc != 'scaler' :
        axs[0].scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,0],label=cc,alpha=0.5)
axs[0].legend()

axs[1].scatter(scaled_web_data[degree]['data'][:,0], scaled_web_data[degree]['data'][:,0], label='websites',alpha=0.5)
axs[1].legend()

for cc in scaled_vals[degree]:
    if cc != 'scaler' :
        axs[2].scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,0],label=cc,alpha=0.5)
axs[2].legend()

axs[2].scatter(scaled_web_data[degree]['data'][:,0], scaled_web_data[degree]['data'][:,0], label='websites',alpha=0.5)
axs[2].legend()


In [ ]:
# degree = 1
# web_data[degree]  = get_website_by_degree(degree)
# mm_scaled_web_data[degree] = {'labels':web_data[degree]['labels'],
#                               'data':(web_data[degree]['data']-mm_scaled_vals[degree]['min_data'])/(mm_scaled_vals[degree]['max_data']-mm_scaled_vals[degree]['min_data'])}

# fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(12,5))
# for cc in mm_scaled_vals[degree]:
#     if cc not in not_cc:
#         axs[0].scatter(mm_scaled_vals[degree][cc][:,0],mm_scaled_vals[degree][cc][:,0],label=cc,alpha=0.5)
# axs[0].legend()

# axs[1].scatter(mm_scaled_web_data[degree]['data'][:,0], mm_scaled_web_data[degree]['data'][:,0], label='websites',alpha=0.5)
# axs[1].legend()

# for cc in mm_scaled_vals[degree]:
#     if cc not in not_cc :
#         axs[2].scatter(mm_scaled_vals[degree][cc][:,0],mm_scaled_vals[degree][cc][:,0],label=cc,alpha=0.5)
# axs[2].legend()

# axs[2].scatter(mm_scaled_web_data[degree]['data'][:,0], mm_scaled_web_data[degree]['data'][:,0], label='websites',alpha=0.5)
# axs[2].legend()


In [ ]:
# degree = 2
# mm_scaled_vals[degree]['combined'] = []
# for cc in ['dctcp','highspeed','lp','reno']:
#     mm_scaled_vals[degree]['combined'] += list(mm_scaled_vals[degree][cc])
# mm_scaled_vals[degree]['combined'] = np.array(mm_scaled_vals[degree]['combined'])

In [ ]:
degree = 2
scaled_vals[degree]['combined'] = []
for cc in ['dctcp','highspeed','lp','reno']:
    scaled_vals[degree]['combined'] += list(scaled_vals[degree][cc])
scaled_vals[degree]['combined'] = np.array(scaled_vals[degree]['combined'])

In [ ]:
# degree = 2
# web_data[degree]  = get_website_by_degree(degree)
# mm_scaled_web_data[degree] = {'labels':web_data[degree]['labels'],
#                               'data':(web_data[degree]['data']-mm_scaled_vals[degree]['min_data'])/(mm_scaled_vals[degree]['max_data']-mm_scaled_vals[degree]['min_data'])}

# fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(12,5))
# for cc in mm_scaled_vals[degree]:
#     if cc not in not_cc:
#         axs[0].scatter(mm_scaled_vals[degree][cc][:,0],mm_scaled_vals[degree][cc][:,1],label=cc,alpha=0.5)
# axs[0].legend()

# axs[1].scatter(mm_scaled_web_data[degree]['data'][:,0], mm_scaled_web_data[degree]['data'][:,1], label='websites',alpha=0.5)
# axs[1].legend()

# for cc in mm_scaled_vals[degree]:
#     if cc not in not_cc :
#         axs[2].scatter(mm_scaled_vals[degree][cc][:,0],mm_scaled_vals[degree][cc][:,1],label=cc,alpha=0.5)
# axs[2].legend()

# axs[2].scatter(mm_scaled_web_data[degree]['data'][:,0], mm_scaled_web_data[degree]['data'][:,1], label='websites',alpha=0.5)
# axs[2].legend()


In [ ]:
degree = 2
web_data[degree]  = get_website_by_degree(degree)
scaled_web_data[degree] = {'labels':web_data[degree]['labels'], 'data':np.array(scaled_vals[degree]['scaler'].transform(web_data[degree]['data']))}

fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(12,5))
for cc in scaled_vals[degree]:
    if cc != 'scaler' :
        axs[0].scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,1],label=cc,alpha=0.5)
axs[0].legend()

axs[1].scatter(scaled_web_data[degree]['data'][:,0], scaled_web_data[degree]['data'][:,1], label='websites',alpha=0.5)
axs[1].legend()

for cc in scaled_vals[degree]:
    if cc != 'scaler' :
        axs[2].scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,1],label=cc,alpha=0.5)
axs[2].legend()

axs[2].scatter(scaled_web_data[degree]['data'][:,0], scaled_web_data[degree]['data'][:,1], label='websites',alpha=0.5)
axs[2].legend()


In [ ]:
# degree = 2
# fig = plt.figure(figsize=(10,5))
# for cc in mm_scaled_vals[degree]:
#     if cc not in not_cc:
#         if cc != 'cubicQ' and cc != "htcp": 
            
#             plt.scatter(mm_scaled_vals[degree][cc][:,0],mm_scaled_vals[degree][cc][:,1],label=cc,alpha=0.5)
# plt.legend()
# plt.show()

In [ ]:
degree = 2
fig = plt.figure(figsize=(10,5))
for cc in scaled_vals[degree]:
    if cc != 'scaler':
        if cc != 'cubicQ' and cc != "htcp":
            plt.scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,1],label=cc,alpha=0.5)
plt.legend()
plt.show()

Combining dctcp, highspeed, lp, reno and westwood

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# degree = 3
# web_data[degree]  = get_website_by_degree(degree)
# mm_scaled_web_data[degree] = {'labels':web_data[degree]['labels'],
#                               'data':(web_data[degree]['data']-mm_scaled_vals[degree]['min_data'])/(mm_scaled_vals[degree]['max_data']-mm_scaled_vals[degree]['min_data'])}

# fig = plt.figure(figsize=(15,8))
# ax = fig.add_subplot(111, projection='3d')

# for cc in mm_scaled_vals[degree]:
#     if cc not in not_cc :
#         ax.scatter(mm_scaled_vals[degree][cc][:,0],mm_scaled_vals[degree][cc][:,1], mm_scaled_vals[degree][cc][:,2], label=cc,alpha=0.5)
# ax.legend()


# fig = plt.figure(figsize=(15,8))
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(mm_scaled_web_data[degree]['data'][:,0], mm_scaled_web_data[degree]['data'][:,1], mm_scaled_web_data[degree]['data'][:,2],  label='websites',alpha=0.5)
# ax.legend()


# fig = plt.figure(figsize=(15,8))
# ax = fig.add_subplot(111, projection='3d')

# for cc in mm_scaled_vals[degree]:
#     if cc not in not_cc :
#         ax.scatter(mm_scaled_vals[degree][cc][:,0],mm_scaled_vals[degree][cc][:,1],mm_scaled_vals[degree][cc][:,2],label=cc,alpha=0.5)
# ax.legend()

# ax.scatter(mm_scaled_web_data[degree]['data'][:,0], mm_scaled_web_data[degree]['data'][:,1],mm_scaled_web_data[degree]['data'][:,2], label='websites',alpha=0.5)
# ax.legend()


In [ ]:
degree = 3
web_data[degree]  = get_website_by_degree(degree)
scaled_web_data[degree] = {'labels':web_data[degree]['labels'], 'data':np.array(scaled_vals[degree]['scaler'].transform(web_data[degree]['data']))}

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111, projection='3d')

for cc in scaled_vals[degree]:
    if cc != 'scaler' :
        ax.scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,1], scaled_vals[degree][cc][:,2], label=cc,alpha=0.5)
ax.legend()


fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(scaled_web_data[degree]['data'][:,0], scaled_web_data[degree]['data'][:,1], scaled_web_data[degree]['data'][:,2],  label='websites',alpha=0.5)
ax.legend()


fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111, projection='3d')

for cc in scaled_vals[degree]:
    if cc != 'scaler' :
        ax.scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,1], scaled_vals[degree][cc][:,2],label=cc,alpha=0.5)
ax.legend()

ax.scatter(scaled_web_data[degree]['data'][:,0], scaled_web_data[degree]['data'][:,1],scaled_web_data[degree]['data'][:,2], label='websites',alpha=0.5)
ax.legend()


### Detecting outliers (different thresholds for different degree)

In [ ]:
def calc_thresh(thresh,degree):
    return np.array(degree*[thresh])    

In [ ]:
# def calc_thresh(degree,weight=[1]):
#     total_sum = 0
#     total_len = 0
#     for cc in scaled_vals[degree]:
#         if cc in not_cc:
#             continue
#         mn = np.mean(scaled_vals[degree][cc])
# #         print(mn)
#         dist = weight*(abs(scaled_vals[degree][cc]-mn))
#         dist = np.sqrt(np.sum((dist**2),axis=1))
# #         print(dist)
#         total_sum += np.sum(dist)
#         total_len += len(dist)
# #         print(total_sum,total_len)
#     return total_sum/total_len
        

In [ ]:
def get_non_outliers(degree,thresh,weight=[1]):
    non_outliers_index = []
    
    for cc in scaled_vals[degree]:
        if cc == 'scaler':
            continue
        for i in range(len(x)):
            isNo = 1
            for d in range(degree):
                if abs(x[i][d]) > thresh:
                    isNo = 0
            if isNo:
                non_outliers_index.append(i)
    non_outliers_index = list(set(non_outliers_index))
    return non_outliers_index

In [ ]:
# def get_non_outliers(degree,weight=[1]):
#     non_outliers_index = []
#     for cc in scaled_vals[degree]:
#         if cc in not_cc:
#             continue
#         mn = np.mean(scaled_vals[degree][cc],axis=0)
#         dist = weight*(abs(x-mn))
#         dist = np.sqrt(np.sum(dist**2,axis=1))
#         for i in range(len(x)):
#             print(dist[i],dist[i]<thresh)
#             if dist[i] < thresh:
#                 non_outliers_index.append(i)
#     non_outliers_index = list(set(non_outliers_index))
#     return non_outliers_index

In [ ]:
def plot_index(out,l,co='b'):
    n_rows = math.ceil(len(out)/3)
    if n_rows == 0:
        return
    fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
    c=0
    if n_rows == 1:
        for j in range(3):
            if c < len(out):
                ax[j].plot(web_mp[l[out[c]]]['time'],web_mp[l[out[c]]]['data'],color=co)
                ax[j].set_title(label=l[out[c]])
                c+=1
        return
        
    for i in range(n_rows):
        for j in range(3):
            if c < len(out):
                ax[i][j].plot(web_mp[l[out[c]]]['time'],web_mp[l[out[c]]]['data'],color=co)
                ax[i][j].set_title(label=l[out[c]])
                c+=1


In [ ]:
usable_web = {}

In [ ]:
degree = 1
thresh = calc_thresh(degree,3)
print("Thresh", thresh)
non_outliers_index = []
x = scaled_web_data[degree]['data']
l = scaled_web_data[degree]['labels']
print("Total",len(l))
non_outliers_index = get_non_outliers(degree,3)
print("Non outliers", len(non_outliers_index))
fig = plt.figure(figsize=(7,7))
for cc in scaled_vals[degree]:
    if cc=='scaler':
        continue
    plt.scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,0],label=cc,alpha=0.5)
pts = np.array([scaled_web_data[degree]['data'][i] for i in non_outliers_index])
plt.scatter(pts[:,0], pts[:,0],label='web',alpha=0.5)
plt.legend()

plot_index(non_outliers_index,l)
# n_rows = math.ceil(len(non_outliers_index)/3)
# fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
# c=0
# for i in range(n_rows):
#     for j in range(3):
#         if c < len(non_outliers_index):
#             ax[i][j].plot(web_mp[l[non_outliers_index[c]]]['time'],web_mp[l[non_outliers_index[c]]]['data'],label=l[c])
#             c+=1
# plt.legend()
outliers_index = [i for i in range(len(l)) if i not in non_outliers_index]
print("Outliers",len(outliers_index))
plot_index(outliers_index,l,'r')

# n_rows = math.ceil(len(outliers_index)/3)
# fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
# c = 0
# for i in range(n_rows):
#     for j in range(3):
#         if c < len(l):
#             ax[i][j].plot(web_mp[l[c]]['time'],web_mp[l[c]]['data'],color='r')
#             c+=1
usable_web[degree] = non_outliers_index

In [ ]:
degree = 2
thresh = calc_thresh(degree,3)
print("Thresh", thresh)
non_outliers_index = []
x = scaled_web_data[degree]['data']
l = scaled_web_data[degree]['labels']
print("Total",len(l))
non_outliers_index = get_non_outliers(degree,15,[0.67,0.33])
print("Non outliers", len(non_outliers_index))

fig = plt.figure(figsize=(7,7))
for cc in scaled_vals[degree]:
    if cc=='scaler':
        continue
    if cc in ['reno','lp','dctcp','highspeed']:
        continue
    plt.scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,1],label=cc,alpha=0.5)
pts = np.array([scaled_web_data[degree]['data'][i] for i in non_outliers_index])
if len(pts) > 0:
    plt.scatter(pts[:,0], pts[:,1],label='web',alpha=0.5)
plt.legend()
plt.show()

plot_index(non_outliers_index,l)
# n_rows = math.ceil(len(non_outliers_index)/3)
# fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
# c=0
# for i in range(n_rows):
#     for j in range(3):
#         if c < len(non_outliers_index):
#             ax[i][j].plot(web_mp[l[non_outliers_index[c]]]['time'],web_mp[l[non_outliers_index[c]]]['data'],label=l[c])
#             c+=1
# plt.legend()
outliers_index = [i for i in range(len(l)) if i not in non_outliers_index]
print("Outliers",len(outliers_index))
plot_index(outliers_index,l,'r')
# n_rows = math.ceil(len(outliers_index)/3)
# fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
# c = 0
# for i in range(n_rows):
#     for j in range(3):
#         if c < len(l):
#             ax[i][j].plot(web_mp[l[c]]['time'],web_mp[l[c]]['data'],color='r')
#             c+=1
usable_web[degree] = non_outliers_index

In [ ]:
name = "inquirer.net" 
new_name = "../../aws/website_measurements/26_1k_cut/mumbai/" + name + "-0-50-200-2-tcp.csv"
plot_one_bt(new_name,p="y",t=1)

In [ ]:
degree = 3
thresh = calc_thresh(degree,3)
print("Thresh", thresh)
non_outliers_index = []
x = scaled_web_data[degree]['data']
l = scaled_web_data[degree]['labels']
print("Total",len(l))

non_outliers_index = get_non_outliers(degree,3,[0.50,0.33,0.17])

print("Non outliers", len(non_outliers_index))

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111, projection='3d')

for cc in scaled_vals[degree]:
    if cc=='scaler':
        continue
    ax.scatter(scaled_vals[degree][cc][:,0],scaled_vals[degree][cc][:,1],scaled_vals[degree][cc][:,2],label=cc,alpha=0.5)
pts = np.array([scaled_web_data[degree]['data'][i] for i in non_outliers_index])
ax.scatter(pts[:,0], pts[:,1], pts[:,2],label='web',alpha=0.5)
ax.legend()
plt.show()
plot_index(non_outliers_index,l)
# n_rows = math.ceil(len(non_outliers_index)/3)
# fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
# c=0
# for i in range(n_rows):
#     for j in range(3):
#         if c < len(non_outliers_index):
#             ax[i][j].plot(web_mp[l[non_outliers_index[c]]]['time'],web_mp[l[non_outliers_index[c]]]['data'],label=l[c])
#             c+=1
# plt.legend()
outliers_index = [i for i in range(len(l)) if i not in non_outliers_index]
print("Outliers",len(outliers_index))
plot_index(outliers_index,l,'r')
# n_rows = math.ceil(len(outliers_index)/3)
# fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
# c = 0
# for i in range(n_rows):
#     for j in range(3):
#         if c < len(l):
#             ax[i][j].plot(web_mp[l[c]]['time'],web_mp[l[c]]['data'],color='r')
#             c+=1
usable_web[degree] = non_outliers_index

## Now applying gaussian model on all the website data and getting the best probability

In [ ]:
scaled_vals[2].pop('combined')

In [ ]:
def create_test_data(degree):
    data = []
    labels = []
    for i in usable_web[degree]:
        data.append(scaled_web_data[degree]['data'][i])
        labels.append(scaled_web_data[degree]['labels'][i])
    return data,labels

In [ ]:
def create_train_data(degree):
    X = []
    y = []
    count_to_map = {}
    count = 1
    for cc in scaled_vals[degree]:
        if cc != 'scaler':
            X += list(scaled_vals[degree][cc])
            y += [count]*len(scaled_vals[degree][cc])
            count_to_map[count] = cc
            count+=1
    return X, y, count_to_map

In [ ]:
X = {}
y = {}
count_to_mp = {}

In [ ]:
for d in [1,2,3]:
    X[d], y[d], count_to_mp[d] = create_train_data(d)

In [ ]:
data_test = {}
for i in [1,2,3]:
    data_test[i] = {}
    data_test[i]['data'], data_test[i]['labels'] = create_test_data(i)

In [ ]:
def plot_test(out,l,predict_cc, co='b'):
    n_rows = math.ceil(len(out)/3)
    if n_rows == 0:
        return
    fig, ax = plt.subplots(n_rows,3,figsize=(10,3*n_rows))
    c=0
    if n_rows == 1:
        for j in range(3):
            if c < len(out):
                ax[j].plot(web_mp[l[out[c]]]['time'],web_mp[l[out[c]]]['data'],color=co)
                ax[j].set_title(label = predict_cc[c] + " " + l[out[c]])
                c+=1
        return
        
    for i in range(n_rows):
        for j in range(3):
            if c < len(out):
                ax[i][j].plot(web_mp[l[out[c]]]['time'],web_mp[l[out[c]]]['data'],color=co)
                ax[i][j].set_title(label = predict_cc[c] + " " + l[out[c]])
                c+=1


In [ ]:
# To ADD :
# Count and prior probability check
# Result storing and showing on the graph

In [ ]:
classifiers = {}

In [ ]:
degree = 3
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
X_t = np.array(X[degree])
y_t = np.array(y[degree])
clf.fit(X_t,y_t)
classifiers[degree] = clf
estimates = clf.predict(data_test[degree]['data'])
prob = clf.predict_proba(data_test[degree]['data'])
inde = [i for i in range(len(data_test[degree]['data']))]
cc_predict = [count_to_mp[degree][i] for i in estimates]
plot_test(inde,data_test[degree]['labels'],cc_predict)


In [ ]:
cc_count = {}
for cc in cc_predict:
    if cc not in cc_count:
        cc_count[cc] = 0
    cc_count[cc]+=1
print(cc_count)

In [ ]:
degree = 2
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
X_t = np.array(X[degree])
y_t = np.array(y[degree])
clf.fit(X_t,y_t)
classifiers[degree] = clf
estimates = clf.predict(data_test[degree]['data'])
prob = clf.predict_proba(data_test[degree]['data'])
inde = [i for i in range(len(data_test[degree]['data']))]
cc_predict = [count_to_mp[degree][i] for i in estimates]
plot_test(inde,data_test[degree]['labels'],cc_predict)


In [ ]:
degree = 1
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
X_t = np.array(X[degree])
y_t = np.array(y[degree])
clf.fit(X_t,y_t)
classifiers[degree] = clf
estimates = clf.predict(data_test[degree]['data'])
prob = clf.predict_proba(data_test[degree]['data'])
inde = [i for i in range(len(data_test[degree]['data']))]
cc_predict = [count_to_mp[degree][i] for i in estimates]
plot_test(inde,data_test[degree]['labels'],cc_predict)


In [ ]:
degree = 1
clf = classifiers[degree]
estimates = clf.predict(data_test[degree]['data'])
prob = clf.predict_proba(data_test[degree]['data'])
inde = [i for i in range(len(data_test[degree]['data']))]
cc_predict = [count_to_mp[degree][i] for i in estimates]
plot_test(inde,data_test[degree]['labels'],cc_predict)


In [ ]:
with open("../analysis/final/classifiers.txt","wb") as f:
    pickle.dump(classifiers,f)

In [ ]:
with open("../analysis/final/count_to_mp.txt","wb") as f:
    pickle.dump(count_to_mp,f)

In [ ]:
classifiers

## Simultaneously using a model shape from the websites to help use Dynamic Time Wraping

In [ ]:
step = 0.05
n_t = []
for i in range(201):
    n_t.append(round(step*i,2))

In [ ]:
coeff_best = {}

In [ ]:
for cc in vals_cubicQ:
    deg = cc_degree[cc]
    coeff_cc = list(np.mean(vals_cubicQ[cc][1],axis=0))
    print(cc)
    print(coeff_cc)
    coeff_cc.append(0)
    y_cc = np.polyval(coeff_cc,n_t)
    if deg not in coeff_best:
        coeff_best[deg] = {}
    coeff_best[deg][cc] = y_cc
    plt.plot(n_t,y_cc)
    plt.show()

In [ ]:
coeff_best

In [ ]:
!pip install fastdtw

In [ ]:


a = np.random.random(100)
b = np.random.random(200)

distance, path = fastdtw(a, b, dist=euclidean)
print(f"DTW Distance: {distance}")

In [ ]:
web_cc_mp

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np

a = np.random.random(100)
b = np.random.random(200)

distance, path = fastdtw(a, b)
print(f"DTW Distance: {distance}")

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np

web_dtw = {}
for web in web_mp:
    dist = []
    cc_ts = []
    test_ts = web_mp[web]['data']
    if web in web_cc_mp:
        deg = web_cc_mp[web]['deg']
    else:
        continue
    for cc in coeff_best[deg]:
        cc_ts.append(cc)
        orig_ts = coeff_best[deg][cc]
#         print(test_ts)
#         print(orig_ts)
        distance, path = fastdtw(test_ts, orig_ts)
        dist.append(distance)
    if deg not in web_dtw:
        web_dtw[deg] = {}
    web_dtw[deg][web] = {
        'cc' : cc_ts,
        'dist' : dist
    }

In [ ]:
web_dtw

In [ ]:
for deg in web_dtw:
    for web in web_dtw[deg]:
        min_ind = web_dtw[deg][web]['dist'].index(min(web_dtw[deg][web]['dist']))
        cc_ind =  web_dtw[deg][web]['cc'][min_ind]b
        plt.plot(web_mp[web]['time'], web_mp[web]['data'])
        plt.title(cc_ind+" "+web)
        plt.show()

In [ ]:
fast_web_dtw = web_dtw

In [ ]:
!pip install dtw-python

In [ ]:
from dtw import *
import numpy as np

web_dtw = {}
for web in web_mp:
    dist = []
    cc_ts = []
    test_ts = web_mp[web]['data']
    if web in web_cc_mp:
        deg = web_cc_mp[web]['deg']
    else:
        continue
    for cc in coeff_best[deg]:
        cc_ts.append(cc)
        orig_ts = coeff_best[deg][cc]
#         print(test_ts)
#         print(orig_ts)
        alignment = dtw(test_ts,orig_ts,dist_method=euclidean)
#         plt.plot(alignment.index1, alignment.index2)  
#         plt.show()
        dist.append(alignment.distance)
    if deg not in web_dtw:
        web_dtw[deg] = {}
    web_dtw[deg][web] = {
        'cc' : cc_ts,
        'dist' : dist
    }

In [ ]:
cc_count = {}
for deg in web_dtw:
    for web in web_dtw[deg]:
        min_ind = web_dtw[deg][web]['dist'].index(min(web_dtw[deg][web]['dist']))
        cc_ind =  web_dtw[deg][web]['cc'][min_ind]
        if cc_ind not in cc_count:
            cc_count[cc_ind]=0
        cc_count[cc_ind]+=1
        print(web_dtw[deg][web]['dist'])
        plt.plot(web_mp[web]['time'], web_mp[web]['data'])
        plt.title(cc_ind+" "+web)
        plt.show()

In [ ]:
print(cc_count)

In [ ]:
for deg in web_dtw:
    for web in web_dtw[deg]:
        min_ind = web_dtw[deg][web]['dist'].index(min(web_dtw[deg][web]['dist']))
        cc_ind =  web_dtw[deg][web]['cc'][min_ind]
        print(web_dtw[deg][web]['dist'])
        plt.plot(web_mp[web]['time'], web_mp[web]['data'])
        plt.title(cc_ind+" "+web)
        plt.show()

In [ ]:
# DEGREE 3
x_train = []
y_train = []
count = 0
count_cc_mp = {}
for cc in cc_degree:
    if cc == 'nv':
        continue
    if cc_degree[cc] == 3:
        length = len(vals_cubicQ[cc][1])
        count_cc_mp[count]=cc
        y_t = [count] * length
        x_t = [list(arr) for arr in vals_cubicQ[cc][1]]
        x_train += x_t
        y_train += y_t
        count+=1

print(x_train[0:5])
print(y_train[0:5])

In [ ]:
count_cc_mp

In [ ]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-1, 10],
                     'C': [1, 10, 100, 500,1000,2000]},
                    ]

In [ ]:
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(x_train, y_train)

In [ ]:
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
y_pred = final_model.predict(x_train)
for i in range(len(x_train)):
    print(count_cc_mp[y_train[i]], count_cc_mp[y_pred[i]])

In [ ]:
web_train = []
website = []
for web in cc_mp:
    if cc_mp[web]['deg'] == 3:
        web_train.append(list(cc_mp[web]['coeff'][0:3]))
        website.append(web)
print(web_train[0:5])

In [ ]:
pred_web = final_model.predict(web_train)
# pred_prob = final_model.predict_proba(web_train)
pred_cc = [count_cc_mp[i] for i in pred_web]

In [ ]:
for i in range(0,4):
    print(count_cc_mp[i], list(pred_web).count(i))

In [ ]:
# Create a 3D figure
fig = plt.figure(figsize=(15,8))

# Create a 3D axis
ax = fig.add_subplot(111, projection='3d')

count = 0 
for web in cc_mp:
    if cc_mp[web]['deg'] == 3:
        x_points = []
        y_points = []
        z_points = []
        points = cc_mp[web]['coeff']
        x_points.append(points[0])
        y_points.append(points[1])
        z_points.append(points[2])
        x_points = np.array(x_points)
        y_points = np.array(y_points)
        z_points = np.array(z_points)
        ax.scatter(x_points, y_points, z_points, label = ,color='k')

for cc in cc_degree:
    if cc == "nv" :
        continue
    if cc_degree[cc] == 3:
        x_points = []
        y_points = []
        z_points = []
        for points in vals_cubicQ[cc][1]:
            x_points.append(points[0])
            y_points.append(points[1])
            z_points.append(points[2])
        x_points = np.array(x_points)
        y_points = np.array(y_points)
        z_points = np.array(z_points)
        ax.scatter(x_points, y_points, z_points, label=cc)
    plt.legend()

        
#     plt.legend()
# Plot the 3D surface
# surf = ax.plot_surface(x, y, z, cmap='viridis')

# Add a color bar which maps values to colors
# fig.colorbar(surf)

# Set axis labels
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

# Set a title
plt.title('3D Surface Plot')


plt.show()

# Lets test for the Quadratic Case which will be visually discernable

In [ ]:
# DEGREE 2
x_train = []
y_train = []
count_cc_mp = {}
for cc in cc_degree:
    if cc_degree[cc] == 2:
        if cc == "westwood":
            count=1 
        elif cc == "cubicQ":
            count=3
        else :
            count=2
        length = len(vals_cubicQ[cc][1])
        count_cc_mp[count]=cc
        y_t = [count] * length
        x_t = [list(arr) for arr in vals_cubicQ[cc][1]]
        x_train += x_t
        y_train += y_t
        count+=1

print(x_train[0:5])
print(y_train[0:5])

In [ ]:
# params_grid = [{'kernel': ['rbf','poly'], 'gamma': [1e-1, 10],
#                      'C': [1, 10, 100, 500,1000,2000]},
#                     ]
# svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model = SVC(gamma=100,C=100)
fit_model = svm_model.fit(x_train, y_train)

In [ ]:
# print('Best score for training data:', svm_model.best_score_,"\n") 

# # View the best parameters for the model found using grid search
# print('Best C:',svm_model.best_estimator_.C,"\n") 
# print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
# print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

# final_model = svm_model.best_estimator_
# y_pred = final_model.predict(x_train)
y_pred = fit_model.predict(x_train)
# for i in range(len(x_train)):
#     print(count_cc_mp[y_train[i]], count_cc_mp[y_pred[i]])

In [ ]:
def make_meshgrid(x, y, h=.002):
    x_min, x_max = x.min() - 0.01, x.max() + 0.01
    y_min, y_max = y.min() -0.01, y.max() + 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
# title for the plots
title = ('Decision surface of linear SVC ')
# Set-up grid for plotting.
X0 = np.array([x_i[0] for x_i in x_train])
X1 = np.array([x_i[1] for x_i in x_train])
xx, yy = make_meshgrid(X0, X1)

plot_contours(ax, fit_model, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=y_train, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_ylabel('y label here')
ax.set_xlabel('x label here')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title(title)
ax.legend()
plt.show()

In [ ]:
web_train = []
website = []
for web in cc_mp:
    if cc_mp[web]['deg'] == 3:
        web_train.append(list(cc_mp[web]['coeff'][0:3]))
        website.append(web)
print(web_train[0:5])

In [ ]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

In [ ]:
import importlib

In [ ]:
importlib.reload(sklearn)

In [ ]:
sklearn.show_versions()

In [ ]:
labels = []
data = []
for cc in cc_degree:
    if cc_degree[cc] == 2:
        for points in vals_cubicQ[cc][1]:
            data.append(list(points))
        labels.append([cc]*len(vals_cubicQ[cc][1]))
# print(data)
# print(labels)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(data)
new_data = scaler.transform(data)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,8))

ax.scatter([i[0] for i in new_data],[i[1] for i in new_data], label = ["cubic","quad"], alpha=0.5)
plt.legend()

#         else :
#             ax.scatter(x_points, y_points, label=cc,alpha=0.5)
#     plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,8))
ax.scatter([i[0] for i in data],[i[1] for i in data],alpha=0.5)
plt.legend()

#         else :
#             ax.scatter(x_points, y_points, label=cc,alpha=0.5)
#     plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,8))
for cc in cc_degree:
#     if cc == 'cubicQ':
#         continue
    if cc_degree[cc] == 2:
        x_points = []
        y_points = []
        for points in vals_cubicQ[cc][1]:
            x_points.append(points[0])
            y_points.append(points[1])
        if cc in ["dctcp", "lp", "reno","highspeed"]:
            color = "k"
            
            ax.scatter(x_points, y_points, label=cc,alpha=0.5, color=color)
        else :
            ax.scatter(x_points, y_points, label=cc,alpha=0.5)
    plt.legend()

#         else :
#             ax.scatter(x_points, y_points, label=cc,alpha=0.5)
#     plt.legend()
plt.show()

# Model Making Again

In [ ]:
# Data 
ccs = []
labels = []
data = []
for cc in cc_degree:
    # choose the degree
    if cc_degree[cc] == 2:
        for points in vals_cubicQ[cc][1]:
            data.append(list(points))
        labels += [cc]*len(vals_cubicQ[cc][1])
        if cc not in ccs:
            ccs.append(cc)
print(len(data))
print(len(labels))

In [ ]:
web_2 = []
for web in web_cc_mp:
    if web_cc_mp[web]['deg'] == 2:
        x_points = []
        y_points = []
        points = web_cc_mp[web]['coeff']
        web_2.append([points[0], points[1]])
web_2 = np.array(web_2)

In [ ]:
labels_as_count = []
labels_to_count = {
    "dctcp" : 1,
    "reno" : 1,
    "lp" : 1,
    "highspeed" : 1,
    "westwood" : 2,
    "cubicQ" : 0 
}
count_to_labels = {}
for l in labels:
    labels_as_count.append(labels_to_count[l])
    if labels_to_count[l] not in count_to_labels.keys():
        count_to_labels[labels_to_count[l]] = l
        

In [ ]:
coeff_to_web = {}
for w in web_cc_mp :
    co = web_cc_mp[w]['coeff']
    coeff_to_web[(co[0],co[1])] = w

In [ ]:
len(labels_as_count)

In [ ]:
#Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data)
scaled_data = scaler.transform(data)
scaled_web_2 = scaler.transform(web_2)

In [ ]:
scaled_data

In [ ]:
# Ploting Normal Vs Scaled
fig, ax = plt.subplots(1,1, figsize=(15,8))

for cc in ccs :
    x_ = []
    y_ = []
    for i in range(len(labels)):
        if labels[i] == cc:
            x_.append(data[i][0])
            y_.append(data[i][1])
    ax.scatter(x_,y_,label=cc, alpha=0.5)

plt.legend()
plt.show()

fig, ax = plt.subplots(1,1, figsize=(15,8))

for cc in ccs :
    x_ = []
    y_ = []
    for i in range(len(labels)):
        if labels[i] == cc:
            x_.append(scaled_data[i][0])
            y_.append(scaled_data[i][1])
    ax.scatter(x_,y_,label=cc, alpha=0.5)

plt.legend()
plt.show()

In [ ]:
# Ploting Normal Vs Scaled
fig, ax = plt.subplots(1,1, figsize=(15,8))

for cc in ccs :
    x_ = []
    y_ = []
    for i in range(len(labels)):
        if labels[i] == cc:
            x_.append(data[i][0])
            y_.append(data[i][1])
    ax.scatter(x_,y_,label=cc, alpha=0.5)


ax.scatter(web_2[:,0],web_2[:,1],color='k', alpha=0.5)


plt.legend()
plt.show()

fig, ax = plt.subplots(1,1, figsize=(15,8))

for cc in ccs :
    x_ = []
    y_ = []
    for i in range(len(labels)):
        if labels[i] == cc:
            x_.append(scaled_data[i][0])
            y_.append(scaled_data[i][1])
    ax.scatter(x_,y_,label=cc, alpha=0.5)

ax.scatter(scaled_web_2[:,0],scaled_web_2[:,1],color='k', alpha=0.5)


plt.legend()
plt.show()

In [ ]:
from scipy import stats

In [ ]:
x_data = [p[0] for p in scaled_data]
y_data = [p[1] for p in scaled_data]
x_data, lt = stats.yeojohnson(x_data)
y_data, lt = stats.yeojohnson(y_data)
yj_data = []
for i in range(len(x_data)):
    yj_data.append([x_data[i],y_data[i]])
yj_data = np.array(yj_data)
fig, ax = plt.subplots(1,1, figsize=(15,8))

for cc in ccs :
    x_ = []
    y_ = []
    for i in range(len(labels)):
        if labels[i] == cc:
            x_.append(x_data[i])
            y_.append(y_data[i])
    ax.scatter(x_,y_,label=cc, alpha=0.5)

plt.legend()
plt.show()


In [ ]:
x_data = [p[0] for p in scaled_data]
y_data = [p[1] for p in scaled_data]
x_data, lt_x = stats.yeojohnson(x_data)
y_data, lt_y = stats.yeojohnson(y_data)
yj_data = []
for i in range(len(x_data)):
    yj_data.append([x_data[i],y_data[i]])
yj_data = np.array(yj_data)
fig, ax = plt.subplots(1,1, figsize=(15,8))

for cc in ccs :
    x_ = []
    y_ = []
    for i in range(len(labels)):
        if labels[i] == cc:
            x_.append(x_data[i])
            y_.append(y_data[i])
    ax.scatter(x_,y_,label=cc, alpha=0.5)

    
web_2_x = [p[0] for p in web_2]
web_2_y = [p[1] for p in web_2]

x_data = stats.yeojohnson(web_2_x, lmbda=lt_x)
y_data = stats.yeojohnson(web_2_y, lmbda=lt_y)
yj_web_2 = []
for i in range(len(x_data)):
    yj_web_2.append([x_data[i],y_data[i]])

yj_web_2 = np.array(yj_web_2)

ax.scatter(yj_web_2[:,0],yj_web_2[:,1],color='k', alpha=0.5)
plt.legend()
plt.show()


In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

names = [
    "Gaussian Process",
    "RBF SVM",
    "Naive Bayes",
    "QDA",
]
classifiers = [
    GaussianProcessClassifier(random_state=42),
    SVC(kernel="linear",random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]
X = np.array(data)
y = np.array(labels_as_count)



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

cm_bright = ListedColormap(["#FF0000", "#00FFFF", "#0000FF"])
cm = plt.cm.RdYlBu
i = 1

figure = plt.figure(figsize=(12, 12))
ax = plt.axes()

ax.scatter(
    X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
)

for name, clf in zip(names, classifiers):
    figure = plt.figure(figsize=(12, 12))
    ax = plt.axes()
#     ax = plt.subplot(1, len(classifiers) + 1,i)

    clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    DecisionBoundaryDisplay.from_estimator(
        clf, X, cmap=cm, alpha=0.8, ax=ax, eps=0.5
    )

    # Plot the training points
    ax.scatter(
        X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_xlim(x_min, x_max)
#     ax.set_ylim(y_min, y_max)
#     ax.set_xticks(())
#     ax.set_yticks(())
#     if ds_cnt == 0:
    ax.set_title(name)
    ax.text(
        x_max - 0.3,
        y_min + 0.3,
        ("%.2f" % score).lstrip("0"),
        size=15,
        horizontalalignment="right",
    )
    i+=1
plt.show()


In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

names = [
    "Gaussian Process",
    "RBF SVM",
    "Naive Bayes",
    "QDA",
]
classifiers = [
    GaussianProcessClassifier(random_state=42),
    SVC(kernel="linear",random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]
X = np.array(scaled_data)
y = np.array(labels_as_count)



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

cm_bright = ListedColormap(["#FF0000", "#00FFFF", "#0000FF"])
cm = plt.cm.RdYlBu
i = 1

figure = plt.figure(figsize=(12, 12))
ax = plt.axes()

ax.scatter(
    X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
)

for name, clf in zip(names, classifiers):
    figure = plt.figure(figsize=(12, 12))
    ax = plt.axes()
#     ax = plt.subplot(1, len(classifiers) + 1,i)

    clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    DecisionBoundaryDisplay.from_estimator(
        clf, X, cmap=cm, alpha=0.8, ax=ax, eps=0.5
    )

    # Plot the training points
    ax.scatter(
        X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_xlim(x_min, x_max)
#     ax.set_ylim(y_min, y_max)
#     ax.set_xticks(())
#     ax.set_yticks(())
#     if ds_cnt == 0:
    ax.set_title(name)
    ax.text(
        x_max - 0.3,
        y_min + 0.3,
        ("%.2f" % score).lstrip("0"),
        size=15,
        horizontalalignment="right",
    )
    i+=1
plt.show()


In [ ]:
from sklearn.mixture import GaussianMixture

X = np.array(scaled_data)
y = np.array(labels_as_count)
web_data = scaled_web_2
gm = GaussianMixture(n_components = 3, covariance_type = 'full', random_state=0, )
gm.fit(X)

plt.figure(figsize=(10,20))
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.scatter(X[:,0],X[:,1],c=gm.predict_proba(X)[:,i],cmap='viridis',marker='x')
    plt.scatter(web_data[:,0],web_data[:,1],c=gm.predict_proba(web_data)[:,i],cmap='viridis',marker='x')
    plt.colorbar()
#     plt.show()
prob = gm.predict_proba(web_data)
val = []
# for i in prob :
#     val.append(1-((1-i[0])*(1-i[1])*(1-i[2])))
# plt.plot(val)
# print(val)

In [ ]:
from sklearn.mixture import GaussianMixture

X = np.array(scaled_data)
y = np.array(labels_as_count)
web_data = scaled_web_2
gm = GaussianMixture(n_components = 3, covariance_type = 'full', random_state=0, )
gm.fit(X)

plt.figure(figsize=(10,20))
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.scatter(X[:,0],X[:,1],c=gm.predict_proba(X)[:,i],cmap='viridis',marker='x')
    plt.scatter(web_data[:,0],web_data[:,1],c=gm.predict_proba(web_data)[:,i],cmap='viridis',marker='x')
    plt.colorbar()
#     plt.show()
prob = gm.predict_proba(web_data)
val = []
# for i in prob :
#     val.append(1-((1-i[0])*(1-i[1])*(1-i[2])))
# plt.plot(val)
# print(val)

In [ ]:
def make_ellipses(gmm, ax):
    for n, color in enumerate(colors):
        if gmm.covariance_type == "full":
            covariances = gmm.covariances_[n][:2, :2]
        elif gmm.covariance_type == "tied":
            covariances = gmm.covariances_[:2, :2]
        elif gmm.covariance_type == "diag":
            covariances = np.diag(gmm.covariances_[n][:2])
        elif gmm.covariance_type == "spherical":
            covariances = np.eye(gmm.means_.shape[1]) * gmm.covariances_[n]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
        ell = mpl.patches.Ellipse(
            gmm.means_[n, :2], v[0], v[1], angle=180 + angle, color=color
        )
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
        ax.set_aspect("equal", "datalim")
        
from sklearn.mixture import GaussianMixture
X = np.array(scaled_data)
y = np.array(labels_as_count)

n_classes = 3

estimators = {
    cov_type: GaussianMixture(
        n_components=n_classes, covariance_type=cov_type, max_iter=100, random_state=0
    )
    for cov_type in ["spherical", "diag", "tied", "full"]
}

for index, (name, estimator) in enumerate(estimators.items()):
    # Since we have class labels for the training data, we can
    # initialize the GMM parameters in a supervised manner.
    estimator.means_init = np.array(
        [X_train[y_train == i].mean(axis=0) for i in range(n_classes)]
    )

    # Train the other parameters using the EM algorithm.
    estimator.fit(X_train)

    h = plt.subplot(2, n_estimators // 2, index + 1)
    make_ellipses(estimator, h)

    for n, color in enumerate(colors):
        data = iris.data[iris.target == n]
        plt.scatter(
            data[:, 0], data[:, 1], s=0.8, color=color, label=iris.target_names[n]
        )
    # Plot the test data with crosses
    for n, color in enumerate(colors):
        data = X_test[y_test == n]
        plt.scatter(data[:, 0], data[:, 1], marker="x", color=color)

    y_train_pred = estimator.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel()) * 100
    plt.text(0.05, 0.9, "Train accuracy: %.1f" % train_accuracy, transform=h.transAxes)

    y_test_pred = estimator.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel()) * 100
    plt.text(0.05, 0.8, "Test accuracy: %.1f" % test_accuracy, transform=h.transAxes)

    plt.xticks(())
    plt.yticks(())
    plt.title(name)

plt.legend(scatterpoints=1, loc="lower right", prop=dict(size=12))


plt.show()
gm = GaussianMixture(n_components = 3, covariance_type = 'full', random_state=0, )
gm.fit(X_train)

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

names = [
    "Gaussian Process",
    "RBF SVM",
    "Naive Bayes",
    "QDA",
]
classifiers = [
    GaussianProcessClassifier(random_state=42),
    SVC(kernel="linear",random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]
X = np.array(yj_data)
y = np.array(labels_as_count)



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

cm_bright = ListedColormap(["#FF0000", "#00FFFF", "#0000FF"])
cm = plt.cm.RdYlBu
i = 1

figure = plt.figure(figsize=(12, 12))
ax = plt.axes()

ax.scatter(
    X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
)

for name, clf in zip(names, classifiers):
    figure = plt.figure(figsize=(12, 12))
    ax = plt.axes()
#     ax = plt.subplot(1, len(classifiers) + 1,i)

    clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    DecisionBoundaryDisplay.from_estimator(
        clf, X, cmap=cm, alpha=0.8, ax=ax, eps=0.5
    )

    # Plot the training points
    ax.scatter(
        X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_xlim(x_min, x_max)
#     ax.set_ylim(y_min, y_max)
#     ax.set_xticks(())
#     ax.set_yticks(())
#     if ds_cnt == 0:
    ax.set_title(name)
    ax.text(
        x_max - 0.3,
        y_min + 0.3,
        ("%.2f" % score).lstrip("0"),
        size=15,
        horizontalalignment="right",
    )
    i+=1
plt.show()


In [ ]:
web_2 = []
for web in web_cc_mp:
    if web_cc_mp[web]['deg'] == 2:
        x_points = []
        y_points = []
        points = web_cc_mp[web]['coeff']
        web_2.append([points[0], points[1]])
web_2 = np.array(web_2)

In [ ]:
def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False 

from scipy.stats import chi2

In [ ]:
count_to_labels

In [ ]:
use_vals = []

for i in count_to_labels.keys(): 
    if i in [1,2]:
        continue
    print(i, count_to_labels[i])
    count = i
    X = []
    y = []
    for n in range(len(data)):
        if labels_as_count[n] == i:
            X.append(data[n])
            y.append(labels_as_count[n])
    x = np.array(X)
    y = np.array(y)
    web_data_use = np.array(web_2)
    x_mean = x.mean(axis=0)
    x_covar = np.cov(x, rowvar=False)
    x_covar_inv = np.linalg.inv(x_covar)
    x_diff = web_data_use - x_mean
    md = []
    for i in range(len(x_diff)):
        md.append(x_diff[i].dot(x_covar_inv).dot(x_diff[i]))
    
    p = 1 - chi2.cdf(md, 2)
    ind = []
    thresh = 1000
    
    for i in range(len(md)):
        if md[i] < thresh:
            ind.append(i)
#             print(black)
            print(i,md[i],md[i]<thresh,"-----")
            use_vals.append(i)
        else :
#             print(red)
            print(i,md[i],md[i]<thresh)

fig = plt.figure(figsize = (10,10))
plt.scatter(np.array(data)[:,0], np.array(data)[:,1])
new_data = np.array([web_2[i] for i in set(use_vals)])
plt.scatter(web_2[:,0], web_2[:,1])

            
fig = plt.figure(figsize = (10,10))
plt.scatter(np.array(data)[:,0], np.array(data)[:,1])
new_data = np.array([web_2[i] for i in set(use_vals)])
plt.scatter(new_data[:,0], new_data[:,1])
# for cc in count_to_labels.keys():   
#     temp_check = []
#     for i in range(1,151):
#         temp_check.append(cc+"-"+str(i)+"-0-50-200-2-aws-88-60")
#     cc_mp = get_feature_degree(temp_check, ss=225,p="n",ft_thresh=1,max_deg=cc_degree[cc])
#     x = []
#     for key in cc_mp.keys():
#         x.append(cc_mp[key][0]['coeff'])
#     x = np.array(x)
#     x_mean = x.mean(axis=0)
#     if cc_degree[cc] == 1:
#         x_covar = np.array([[np.cov(x, rowvar=False)]])
#     else :
#         x_covar = np.cov(x, rowvar=False)
#     x_covar_inv = np.linalg.inv(x_covar)
#     print(x_mean)
#     print(x_covar)
#     print(is_pos_def(x_covar))
#     print(is_pos_def(x_covar_inv))
#     x_diff = x - x_mean
#     md = []
#     for i in range(len(x_diff)):
#         md.append(x_diff[i].dot(x_covar_inv).dot(x_diff[i]))
#     p = 1 - chi2.cdf(md, 4)
#     ind = []
#     thresh = 0.05
#     for i in range(len(md)):
#         if p[i] < thresh:
#             print(red)
#             ind.append(i)
#         print(i,md[i],p[i],p[i]<thresh)
#         print(black)
#     dont_use_ind[cc] = ind


In [ ]:
data

In [ ]:
use_vals = []
means = []

for i in count_to_labels.keys(): 
    if i in [0,1]:
        continue
#     print(i, count_to_labels[i])
    count = i
    X = []
    y = []
    for n in range(len(scaled_data)):
        if labels_as_count[n] == i:
            X.append(scaled_data[n])
    x = np.array(X)
    web_data_use = np.array(scaled_web_2)
    x_mean = x.mean(axis=0)
    means.append(x_mean)
#     x_covar = np.cov(x, rowvar=False)
#     x_covar_inv = np.linalg.inv(x_covar)
    x_diff = (web_data_use - x_mean)**2
    md = []
    for row in x_diff:
        md.append(np.sqrt(np.sum(np.dot(np.array([0.66,0.34]),row))))
    thresh = 10
    
    for i in range(len(md)):
        if md[i] < thresh:
#             print(black)
            print(i,md[i],md[i]<thresh,"-----")
            use_vals.append(i)
        else :
#             print(red)
            print(i,md[i],md[i]<thresh)

new_data = np.array([scaled_web_2[i] for i in set(use_vals)])

fig = plt.figure(figsize = (10,10))
plt.scatter(scaled_data[:,0], scaled_data[:,1])
plt.scatter(scaled_web_2[:,0], scaled_web_2[:,1])
plt.scatter(new_data[:,0], new_data[:,1])
plt.scatter(np.array(means)[:,0], np.array(means)[:,1])

# for cc in count_to_labels.keys():   
#     temp_check = []
#     for i in range(1,151):
#         temp_check.append(cc+"-"+str(i)+"-0-50-200-2-aws-88-60")
#     cc_mp = get_feature_degree(temp_check, ss=225,p="n",ft_thresh=1,max_deg=cc_degree[cc])
#     x = []
#     for key in cc_mp.keys():
#         x.append(cc_mp[key][0]['coeff'])
#     x = np.array(x)
#     x_mean = x.mean(axis=0)
#     if cc_degree[cc] == 1:
#         x_covar = np.array([[np.cov(x, rowvar=False)]])
#     else :
#         x_covar = np.cov(x, rowvar=False)
#     x_covar_inv = np.linalg.inv(x_covar)
#     print(x_mean)
#     print(x_covar)
#     print(is_pos_def(x_covar))
#     print(is_pos_def(x_covar_inv))
#     x_diff = x - x_mean
#     md = []
#     for i in range(len(x_diff)):
#         md.append(x_diff[i].dot(x_covar_inv).dot(x_diff[i]))
#     p = 1 - chi2.cdf(md, 4)
#     ind = []
#     thresh = 0.05
#     for i in range(len(md)):
#         if p[i] < thresh:
#             print(red)
#             ind.append(i)
#         print(i,md[i],p[i],p[i]<thresh)
#         print(black)
#     dont_use_ind[cc] = ind


In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
curr_data = np.array(data)
curr_labels = np.array(labels_as_count)
web_2_data = web_2
not_outliers = {}
for i in count_to_labels.keys():    
    print(i, count_to_labels[i])
    count = i
    X = []
    y = []
    for n in range(len(curr_data)):
        if labels_as_count[n] ==  i:
            X.append(curr_data[n])
            y.append(labels_as_count[n])
    X = np.array(X)
    y = np.array(y)
    web_data_use = web_2_data
    ss = StandardScaler()
    X = ss.fit_transform(X)
    web_2_data_use = ss.transform(web_2_data)
    one_class_svm = OneClassSVM(nu=0.001, kernel = 'sigmoid', gamma = 1).fit(X)
    prediction = one_class_svm.predict(web_2_data_use)
    prediction = [1 if i==-1 else 0 for i in prediction]
    temp = []
    for i in range(len(web_2_data)):
        if prediction[i] == 0:
            temp.append(web_2_data[i])
    not_outliers[count] = temp

figure = plt.figure(figsize=(10, 10))
ax = plt.axes()

cm_bright = ListedColormap(["#FF0000","#FFFF00","#0000FF"])
cm = plt.cm.RdYlBu

ax.scatter(
    curr_data[:, 0], curr_data[:, 1], c=curr_labels, cmap=cm_bright, edgecolors="k"
)

ax.scatter(
    web_2_data[:, 0], web_2_data[:, 1], color="k", edgecolors="k"
)

for i in not_outliers:
    if len(not_outliers[i])>0:
        new_data = np.array(not_outliers[i])
        ax.scatter(
            new_data[:, 0], new_data[:, 1], cmap=cm
        )

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
curr_data = np.array(data)
curr_labels = np.array(labels_as_count)
web_2_data = web_2
not_outliers = {}
for i in count_to_labels.keys():    
    print(i, count_to_labels[i])
    count = i
    X = []
    y = []
    for n in range(len(curr_data)):
        if labels_as_count[n] ==  i:
            X.append(curr_data[n])
            y.append(labels_as_count[n])
    X = np.array(X)
    y = np.array(y)
    web_data_use = web_2_data
    ss = StandardScaler()
    X = ss.fit_transform(X)
    web_2_data_use = ss.transform(web_2_data)
    one_class_svm = OneClassSVM(nu=0.001, kernel = 'sigmoid', gamma = 1).fit(X)
    prediction = one_class_svm.predict(web_2_data_use)
    prediction = [1 if i==-1 else 0 for i in prediction]
    temp = []
    for i in range(len(web_2_data)):
        if prediction[i] == 0:
            temp.append(web_2_data[i])
    not_outliers[count] = temp

figure = plt.figure(figsize=(10, 10))
ax = plt.axes()

cm_bright = ListedColormap(["#FF0000","#FFFF00","#0000FF"])
cm = plt.cm.RdYlBu

ax.scatter(
    curr_data[:, 0], curr_data[:, 1], c=curr_labels, cmap=cm_bright, edgecolors="k"
)

ax.scatter(
    web_2_data[:, 0], web_2_data[:, 1], color="k", edgecolors="k"
)

for i in not_outliers:
    if len(not_outliers[i])>0:
        new_data = np.array(not_outliers[i])
        ax.scatter(
            new_data[:, 0], new_data[:, 1], cmap=cm
        )

In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import train_test_split
curr_data = np.array(data)
curr_labels = np.array(labels_as_count)
web_2_data = web_2
not_outliers = {}
for i in count_to_labels.keys():    
    print(i, count_to_labels[i])
    count = i
    X = []
    y = []
    for n in range(len(curr_data)):
        if labels_as_count[n] ==  i:
            X.append(curr_data[n])
            y.append(labels_as_count[n])
    X = np.array(X)
    y = np.array(y)
    ss = StandardScaler()
    X = ss.fit_transform(X)
    web_2_data_use = ss.transform(web_2_data)
    model = EllipticEnvelope(contamination=0.001,random_state=42).fit(X)
    prediction = model.predict(web_2_data_use)
    prediction = [1 if i==-1 else 0 for i in prediction]
    temp = []
    for i in range(len(web_2_data_use)):
        if prediction[i] == 0:
            temp.append(web_2_data_use[i])
    not_outliers[count] = temp

figure = plt.figure(figsize=(10, 10))
ax = plt.axes()

cm_bright = ListedColormap(["#FF0000","#FFFF00","#0000FF"])
cm = plt.cm.RdYlBu

ax.scatter(
    curr_data[:, 0], curr_data[:, 1], c=curr_labels, cmap=cm_bright, edgecolors="k"
)

ax.scatter(
    web_2_data[:, 0], web_2_data[:, 1], color="k", edgecolors="k"
)

for i in not_outliers:
    if len(not_outliers[i])>0:
        print(i)
        new_data = np.array(not_outliers[i])
        color = 'r'
        if i == 1:
            color = 'y'
        elif i == 2:
            color = 'b'
        ax.scatter(
            new_data[:, 0], new_data[:, 1], c=color
        )

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
curr_data = np.array(scaled_data)
curr_labels = np.array(labels_as_count)
web_2_data = scaled_web_2
not_outliers = {}
for i in count_to_labels.keys():    
    print(i, count_to_labels[i])
    count = i
    X = []
    y = []
    for n in range(len(curr_data)):
        if labels_as_count[n] ==  i:
            X.append(curr_data[n])
            y.append(labels_as_count[n])
    X = np.array(X)
    y = np.array(y)
    one_class_svm = OneClassSVM(nu=0.001, kernel = 'rbf', gamma = 100).fit(X)
    prediction = one_class_svm.predict(web_2_data)
    prediction = [1 if i==-1 else 0 for i in prediction]
    temp = []
    for i in range(len(web_2_data)):
        if prediction[i] == 0:
            temp.append(web_2_data[i])
    not_outliers[count] = temp

figure = plt.figure(figsize=(10, 10))
ax = plt.axes()

cm_bright = ListedColormap(["#FF0000","#FFFF00","#0000FF"])
cm = plt.cm.RdYlBu

ax.scatter(
    curr_data[:, 0], curr_data[:, 1], c=curr_labels, cmap=cm_bright, edgecolors="k"
)

ax.scatter(
    web_2_data[:, 0], web_2_data[:, 1], color="k", edgecolors="k"
)
i_to_c = {
    0 : 'r',
    1 : 'y',
    2 : 'b'
}
for i in not_outliers:
    if len(not_outliers[i])>0:
        print(i)
        new_data = np.array(not_outliers[i])
        ax.scatter(
            new_data[:, 0], new_data[:, 1], c=i_to_c[i]
        )

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
curr_data = np.array(yj_data)
curr_labels = np.array(labels_as_count)
web_2_data = yj_web_2
not_outliers = {}
for i in count_to_labels.keys():    
    print(i, count_to_labels[i])
    count = i
    X = []
    y = []
    for n in range(len(curr_data)):
        if labels_as_count[n] ==  i:
            X.append(curr_data[n])
            y.append(labels_as_count[n])
    X = np.array(X)
    y = np.array(y)
    one_class_svm = OneClassSVM(nu=0.001, kernel = 'rbf', gamma = 1000).fit(X)
    prediction = one_class_svm.predict(web_2_data)
    prediction = [1 if i==-1 else 0 for i in prediction]
    temp = []
    for i in range(len(web_2_data)):
        if prediction[i] == 0:
            temp.append(web_2_data[i])
    not_outliers[count] = temp

figure = plt.figure(figsize=(10, 10))
ax = plt.axes()

cm_bright = ListedColormap(["#FF0000","#FFFF00","#0000FF"])
cm = plt.cm.RdYlBu

ax.scatter(
    curr_data[:, 0], curr_data[:, 1], c=curr_labels, cmap=cm_bright, edgecolors="k"
)

ax.scatter(
    web_2_data[:, 0], web_2_data[:, 1], color="k", edgecolors="k"
)

for i in not_outliers:
    if len(not_outliers[i])>0:
        print(count_to_labels[i])
        new_data = np.array(not_outliers[i])
        print(len(new_data))
        ax.scatter(
            new_data[:, 0], new_data[:, 1], c=[i]*len(new_data), cmap=cm_bright
        )

## Instead of this lets use a probability threshold

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

names = [
    "Gaussian Process",
    "RBF SVM",
    "Naive Bayes",
    "QDA",
]
classifiers = [
    GaussianProcessClassifier(random_state=42),
    SVC(kernel="linear",random_state=42,probability=1),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]
X = np.array(scaled_data)
y = np.array(labels_as_count)
data_web_2 = np.array(scaled_web_2)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

cm_bright = ListedColormap(["#FF0000", "#FFFF00", "#0000FF"])
cm = plt.cm.RdYlBu
i = 1

figure = plt.figure(figsize=(8, 8))
ax = plt.axes()

ax.scatter(
    X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
)
ax.scatter(
    data_web_2[:,0], data_web_2[:,1], color='k'
)

for name, clf in zip(names, classifiers):
    figure = plt.figure(figsize=(8, 8))
    ax = plt.axes()
#     ax = plt.subplot(1, len(classifiers) + 1,i)

    clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
#     DecisionBoundaryDisplay.from_estimator(
#         clf, X, cmap=cm, alpha=0.8, ax=ax, eps=0.5
#     )
#     DecisionBoundaryDisplay.from_estimator(
#         clf, scaled_web_2, cmap=cm, alpha=0.8, ax=ax, eps=0.5
#     )
    
    results = clf.predict(data_web_2)
    results_proba = clf.predict_proba(data_web_2)
    # Plot the training points
    ax.scatter(
        X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_xlim(x_min, x_max)
    ax.scatter(
        data_web_2[:, 0],
        data_web_2[:, 1],
        c=results,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_ylim(y_min, y_max)
#     ax.set_xticks(())
#     ax.set_yticks(())
#     if ds_cnt == 0:
    ax.set_title(name)
    ax.text(
        x_max - 0.3,
        y_min + 0.3,
        ("%.2f" % score).lstrip("0"),
        size=15,
        horizontalalignment="right",
    )
    i+=1
plt.show()


In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
    
names = [
    "Gaussian Process",
    "RBF SVM",
    "Naive Bayes",
    "QDA",
    "Decision Tree",
    "Random Forest",
]
classifiers = [
    GaussianProcessClassifier(random_state=42),
    SVC(kernel="linear",random_state=42,probability=1),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(
        max_depth=5, n_estimators=10, max_features=1, random_state=42
    ),
]
X = np.array(yj_data)
y = np.array(labels_as_count)
data_web_2 = np.array(yj_web_2)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

cm_bright = ListedColormap(["#FF0000", "#FFFF00", "#0000FF"])
cm = plt.cm.RdYlBu
i = 1

figure = plt.figure(figsize=(8, 8))
ax = plt.axes()

ax.scatter(
    X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
)
ax.scatter(
    data_web_2[:,0], data_web_2[:,1], color='k'
)

for name, clf in zip(names, classifiers):
    figure = plt.figure(figsize=(8, 8))
    ax = plt.axes()
#     ax = plt.subplot(1, len(classifiers) + 1,i)

#     clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    DecisionBoundaryDisplay.from_estimator(
        clf, X, cmap=cm, alpha=0.8, ax=ax, eps=0.5
    )
#     DecisionBoundaryDisplay.from_estimator(
#         clf, data_web_2, cmap=cm, alpha=0.8, ax=ax, eps=0.5
#     )
    
    results = clf.predict(data_web_2)
    results_proba = clf.predict_proba(data_web_2)
    # Plot the training points
    ax.scatter(
        X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_xlim(x_min, x_max)
    ax.scatter(
        data_web_2[:, 0],
        data_web_2[:, 1],
        c=results,
        cmap=cm_bright,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_ylim(y_min, y_max)
#     ax.set_xticks(())
#     ax.set_yticks(())
#     if ds_cnt == 0:
    ax.set_title(name)
    ax.text(
        x_max - 0.3,
        y_min + 0.3,
        ("%.2f" % score).lstrip("0"),
        size=15,
        horizontalalignment="right",
    )
    i+=1
plt.show()


In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
    
names = [
    "Decision Tree",
#     "Random Forest",
]
classifiers = [
#     GaussianProcessClassifier(random_state=42),
#     SVC(kernel="linear",random_state=42,probability=1),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis(),
    DecisionTreeClassifier(max_depth=10, random_state=42)
#     RandomForestClassifier(
#         max_depth=10, n_estimators=100, random_state=42
#     ),
]
X = np.array(data)
y = np.array(labels_as_count)
data_web_2 = np.array(web_2)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

l_to_c = {
    'cubicQ' : 'red',
    'dctcp' : 'yellow',
    'westwood' : 'blue',
}
cm = ListedColormap(['red','yellow','blue'])
cm_bright = ListedColormap(["#FF0000", "#FFFF00", "#0000FF"])
i = 1

figure = plt.figure(figsize=(8, 8))
ax = plt.axes()

ax.scatter(
    X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm, edgecolors="k"
)
ax.scatter(
    data_web_2[:,0], data_web_2[:,1], color='k'
)

for name, clf in zip(names, classifiers):
    figure = plt.figure(figsize=(8, 8))
    ax = plt.axes()
#     ax = plt.subplot(1, len(classifiers) + 1,i)

#     clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    DecisionBoundaryDisplay.from_estimator(
        clf, X, cmap=cm_bright, alpha=0.8, ax=ax, eps=0.5
    )
#     DecisionBoundaryDisplay.from_estimator(
#         clf, data_web_2, cmap=cm_bright, alpha=0.8, ax=ax, eps=0.5
#     )
    
    results = clf.predict(data_web_2)
    print(results)
    results_proba = clf.predict_proba(data_web_2)
    # Plot the training points
    ax.scatter(
        X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm, edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=cm,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_xlim(x_min, x_max)
    new_c = []
    if len(set(results)) == 2 :
        new_c = ['yellow', 'blue']
        cm = ListedColormap(new_c)
    ax.scatter(
        data_web_2[:, 0],
        data_web_2[:, 1],
        c=results,
        cmap=cm,
        edgecolors="k",
        alpha=0.6,
    )
#     ax.set_ylim(y_min, y_max)
#     ax.set_xticks(())
#     ax.set_yticks(())
#     if ds_cnt == 0:
    ax.set_title(name)
    ax.text(
        x_max - 0.3,
        y_min + 0.3,
        ("%.2f" % score).lstrip("0"),
        size=15,
        horizontalalignment="right",
    )
    i+=1
plt.show()

for i in set(results):
    print("Printing for ", count_to_labels[i])
    for i in range(len(data_web_2)):
        tcoeff = (data_web_2[i][0],data_web_2[i][1])
        web = coeff_to_web[tcoeff]
        tdata = web_mp[web]['data']
        ttime = web_mp[web]['time']
        plt.plot(ttime,tdata)
        plt.show()

In [ ]:
from sklearn.covariance import EllipticEnvelope

model = EllipticEnvelope(contamination=0.01, )

# Clustering using DBSCAN

In [ ]:
X = scaled_data

import numpy as np

from sklearn import metrics
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=5).fit(X)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

unique_labels = set(labels)
figure = plt.figure(figsize=(8, 8))
ax = plt.axes()
for u in unique_labels:
    x = []
    for i in range(len(X)):
        if labels[i] == u:
            x.append(X[i])
    x = np.array(x)
    ax.scatter(
        x[:, 0],
        x[:, 1],
        alpha=0.6,
    )

In [ ]:
X = np.concatenate((data, web_2),axis=0)

import numpy as np

from sklearn import metrics
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.05, min_samples=5).fit(X)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

unique_labels = set(labels)
figure = plt.figure(figsize=(8, 8))
ax = plt.axes()
for u in unique_labels:
    x = []
    for i in range(len(X)):
        if labels[i] == u:
            x.append(X[i])
    x = np.array(x)
    ax.scatter(
        x[:, 0],
        x[:, 1],
        alpha=0.6,
    )

In [ ]:
web_mp

In [ ]:
web_cc_mp